In [1]:
from telethon import TelegramClient, events, sync, Button
from telethon.tl.types import User
from telethon.tl.functions.messages import GetStickerSetRequest
import re  
import requests  
import json
import pandas as pd
from key_generator.key_generator import generate
import urllib.request
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import datetime
from datetime import date
import numpy as np
from gekko import GEKKO
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint

In [3]:
fkn_ids = list(pd.read_csv('fkn2.csv', sep=';')['id'])
student_subjects = []
student_number = 0
for i in tqdm(range(len(fkn_ids))):
    url = 'https://ruz.hse.ru/api/schedule/student/' + str(fkn_ids[i]) + '?start=2021.01.01&finish=2021.06.15'
    ss = requests.get(url).json()
    time.sleep(0.1)
    sss = []
    for i in range(len(ss)):
        sss.append(ss[i]['disciplineOid'])
    sss = list(set(sss))
    student_subjects.append(sss)


  5%|▌         | 123/2457 [02:58<56:31,  1.45s/it]  


KeyboardInterrupt: 

In [4]:
from __future__ import print_function
import datetime
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']



"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    # if creds and creds.expired and creds.refresh_token:
    #     creds.refresh(Request())
    # else:
    flow = InstalledAppFlow.from_client_secrets_file(
        'credentials0.json', SCOPES)
    creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=986666376835-dqjk198rf1fhngi1f2hf8ios8mkipvej.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62917%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=vsKJUkA4n9DSmZ4k5mHvXttIh3q32A&access_type=offline


In [5]:
CREDENTIALS_FILE = 'credentials0.json'

def get_calendar_service():
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               CREDENTIALS_FILE, SCOPES)
           creds = flow.run_local_server(port=0)

       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('calendar', 'v3', credentials=creds)
   return service

In [6]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials2.json',scope)
#create gspread authorize using that credential
client2 = gspread.authorize(creds)

In [7]:
users = pd.DataFrame(columns = ['id_user', 'status'])
state = {}
course = {}
course00 = {}
name = {}
surname = {}
last_name = {}

subject_name_to_id = {}
subject_id_to_name = {}
subject = {} #get dedlines
subject_id = {}
dedline = {} #get (date_start, date_end, difficulty)
max_subject_id = 0
cur_discipline = {}
cur_op_in_teacher = {}
cur_ded = {}
cur_name = {}
cur_pos = {}
cur_stud_name = {}
cur_stud_last_name = {}
cur_stud_surname = {}
name_op = {}
anses = {}

client = TelegramClient(None, 4635643, 'bab7d7b2f0f7830dbf063cda0e07aefa')
await client.start(bot_token='1741873045:AAHJEfiW9iMgCNfB-qCg6OYk8TIkcQTETs4')

users = pd.DataFrame(columns = ['id_user', 'status'])
state = {}





@client.on(events.NewMessage())
async def handler(event):
    global max_subject_id, azaz
    print(event.message.message)
    if event.chat.id not in list(users.id_user) or state[event.chat.id] == 0:
        intro = '''
        Данный бот предназначен для оптимизпции расписания всех форм контроля студента
    Если вы студент, то вам необхоим ключ для входа от вашего университета. Войдя в свою образовательную программу, вы сможете посмотреть дады всех ваших дедлайнов и других форм контроля. Вы также сможете синхронизировать эти даты со своим календарем.
    Если вы преподаватель, то вам также необходим ключ для входа от вашего университета. Зайдя по ключу, введите свои инициалы. Далее, выберите интересующий вас курс и создайте в нем все необходимые формы контроля (контрольные работы, домашние задания итд)
Выполнив этот шаг, добавьте в каждую форму контроля столько дедлайнов, сколько необходимо. Установите порядок дедлайнов, выберите их сложность и необхомые даты. В последующем вы сможете изменить эту информацию.
В конечном итоге, Вы сможете посмотреть расписание всех форм контролей по своим предметам 
    Если вы представитель нового университета, создайте преподавательский и студенческие ключи
        '''
        await client.send_message(event.chat_id, intro)
        keyboard = [
            [  
                Button.inline("Студент", '1'), 
                Button.inline("Преподаватель", '2')
            ],
            [
                Button.inline("Представитель нового университета", '3')
            ]
        ]
        state[event.chat.id] = 0
        await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
        
    elif state[event.chat.id] == 1:
        if event.message.message not in course00:
            state[event.chat.id] = 200
            keyboard = [
                [  
                    Button.inline("Попробовать еще раз", '2'),
                    Button.inline("Назад", '-1')
                ]
            ]
            await client.send_message(event.chat_id, 'Введен неверный ключ', buttons=keyboard)
        else:
            state[event.chat.id] = 201
            await client.send_message(event.chat_id, f'Вы вошли в <{course00[event.message.message]}>. \nВведите Ваше имя.\nЕсли хотите вернуться назад -- введите <Назад>')
            
    
        
    
    elif state[event.chat.id] == 201:
        cur_stud_name[event.chat.id] = event.message.message
        state[event.chat.id] = 202
        await client.send_message(event.chat_id, 'Введите Вашу фамилию')
        
    elif state[event.chat.id] == 202:
        cur_stud_surname[event.chat.id] = event.message.message
        state[event.chat.id] = 203
        await client.send_message(event.chat_id, 'Введите Ваше отчество. При отсутствие напишите <Нет>')
        
    elif state[event.chat.id] == 203:
        cur_stud_last_name[event.chat.id] = event.message.message
        state[event.chat.id] = 204
        dedlines_by_lesson = {}
        tpr_lst = list(subject_id_to_name.keys())
        for i in range(len(subject_id_to_name)):
            if subject_id_to_name[i][1] in dedlines_by_lesson:
                dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])
            else:
                dedlines_by_lesson[subject_id_to_name[i][1]] = []
                dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])


        get_dead = {}
        ccc = 0

        m = GEKKO()
        X = []
        W = []
        D_s = []
        D_e = []
        Students = []
        # x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]


        # m.Equation(x1+x2+x3+x4<=25)




        for i in range(len(subject)):
            qq = list(subject.keys())[i]
            idd = subject_id_to_name[qq][1]
            get_dead[tpr_lst[i]] = []
            for j in range(len(subject[qq])):
                day_start = subject[qq][j][1].split('.')[0]
                month_start = subject[qq][j][1].split('.')[1]
                year_start = subject[qq][j][1].split('.')[2]

                day_end = subject[qq][j][2].split('.')[0]
                month_end = subject[qq][j][2].split('.')[1]
                year_end = subject[qq][j][2].split('.')[2]

                date_start = date(int(year_start),int(month_start),int(day_start))
                date_end = date(int(year_end),int(month_end),int(day_end))

                tpr_date = date(2019,1,1)
                date_start = (date_start - tpr_date).days
                date_end = (date_end - tpr_date).days
                D_s.append(date_start)
                D_e.append(date_end)
                get_dead[tpr_lst[i]].append(ccc)
                ccc+=1
                X.append(m.Var(lb=date_start, ub=date_end))
                W.append(subject[i][j][0])
                if j != 0:
                    m.Equation(X[-1] > X[-2] + 0.4)


        student_deadlines = []
        for i in range(len(student_subjects)):
            tt = []
            for j in range(len(student_subjects[i])):
                try:
                    tt = dedlines_by_lesson[student_subjects[i][j]]
                except:
                    pass
            student_deadlines.append(tt)

        cnt = 0
        cnt2=0
        expr = []

        P = 0
        for stud in range(len(student_deadlines)):
            lessons = []
        #     if cnt > 15:
        #         break
            for i in range(len(student_deadlines[stud])):
                lessons = lessons + get_dead[student_deadlines[stud][i]]
        #         print(get_dead[student_deadlines[stud][i]])
        #     print(stud)
            for i in range(len(lessons)):
                print(stud)
                for j in range(i+1, len(lessons)):
        #             P += (W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2
                    expr.append((W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2)
        #             print((W[lessons[i]] + W[lessons[j]]))
        #             print(np.abs(1 + X[lessons[i]] - X[lessons[j]]))
                    cnt2+=1
            if len(lessons) > 0:
                cnt+=1

        for i in range(len(expr)):
            m.Obj(expr[i])

        m.solve()
        try:
            url_id = 'https://ruz.hse.ru/api/search?term=' + cur_stud_surname[event.chat.id] + '%' + cur_stud_name[event.chat.id] + '%' + cur_stud_last_name[event.chat.id]
            soup_id = requests.get(url_id).json()
            ids_me = []
            for i in range(len(soup_id)):
                if soup_id[i]['type'] == 'student':
                    ids_me.append(soup_id[i]['id'])
            url = 'https://ruz.hse.ru/api/schedule/student/' + ids_me[0] + '?start=2021.01.01&finish=2021.06.11'
            data = requests.get(url).json()
            my_disciplines = {}
            for mini_data in data:
                if mini_data['discipline'] not in list(my_disciplines.values()):
                    my_disciplines[mini_data['disciplineOid']] = mini_data['discipline']

            idx = []
            for less in range(len(subject_id_to_name)):
                if list(subject_id_to_name.values())[less][1] in list(my_disciplines.keys()):
                    idx.append(less)

            kk = list(get_dead.keys())
            vv = list(get_dead.values())

            ans = []
            ppp = 0
            print(idx)
            print(kk)
            print(my_disciplines)
            for i in range(len(kk)):

                ee = 1
                for j in range(len(get_dead[kk[i]])):
                    if kk[i] in idx:
                        ans.append((str(tpr_date + datetime.timedelta(days=X[ppp][0])), my_disciplines[subject_id_to_name[kk[i]][1]], subject_id_to_name[kk[i]][0] + '-' + str(ee)))
                    ppp+=1
                    ee+=1
            print(ans)
            sheet = client2.open('Таблица').sheet1
            pp_ = pprint.PrettyPrinter()
            results = sheet.get_all_records()
            ddf = pd.DataFrame(ans, columns=['0', '1', '2']).sort_values(by='0').values
            for i in range(len(ans)):
                sheet.append_row(list(ddf[i]))
                
            keyboard = [
                [  
                    Button.inline("Хочу синхронизовать расписание дедлайнов с календарем", '2'),
                    Button.inline("Назад", '-1')
                ]
            ]
            
            anses[event.chat_id] = ans
            await client.send_message(event.chat_id, 'Ваше расписание https://docs.google.com/spreadsheets/d/1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU/edit#gid=0', buttons=keyboard)
            
            state[event.chat.id] = 222
            
            
        except:
            state[event.chat.id] = 208
            keyboard = [
                [  
                    Button.inline("Попробовать еще раз", '2'),
                    Button.inline("Назад", '-1')
                ]
            ]
            await client.send_message(event.chat_id, 'Данного студента не сущетствует, либо у него нет никаких дедлайнов', buttons=keyboard)
            
        
    elif state[event.chat.id] == 223: 
        calendar = {
            'summary': 'Календарь расписания дедлайнов',
            'timeZone': 'Europe/Moscow'
        }

        created_calendar = service.calendars().insert(body=calendar).execute()


        calendars_result = service.calendarList().list().execute()

        calendars = calendars_result.get('items', [])

        calendar_id = calendars[-1]['id']

        rule = {
            'scope': {
                'type': 'group',
                'value': event.message.message,
            },
            'role': 'writer'
        }

        created_rule = service.acl().insert(calendarId=calendar_id, body=rule).execute()
        this_ans = anses[event.chat.id]
        for i in range(len(this_ans)):
            my_event = {
              'summary': this_ans[i][1],
              'description': this_ans[i][2],
              'start': {
                'dateTime': this_ans[i][0] + 'T00:00:00-07:00',
                'timeZone': 'Europe/Moscow',
              },
              'end': {
                'dateTime': this_ans[i][0] + 'T12:00:00-07:00',
                'timeZone': 'Europe/Moscow',
              },
              'reminders': {
                'useDefault': False,
                'overrides': [
                  {'method': 'email', 'minutes': 24 * 60},
                  {'method': 'popup', 'minutes': 10},
                ],
              },
            }

            service.events().insert(calendarId=calendar_id, body=my_event).execute()
            
        keyboard = [
                [  
                    Button.inline("Назад", '-1')
                ]
            ]
        state[event.chat.id] = -1
        #тут ссылку немного другую надо
        await client.send_message(event.chat_id, 'Ссылка на Ваш календарь\n' + calendar_id, buttons=keyboard)
            

        
        
        
        
    
    elif state[event.chat.id] == 3:
        await client.send_message(event.chat_id, 'Ваш ключ, который необходимо отправить преподавателям:')
        key = generate()
        keyboard = [
            [  
                Button.inline("Назад", 'd')
            ]
        ]
        state[event.chat.id] = -1
        course[key.get_key()] = event.message.message
        
        await client.send_message(event.chat_id, key.get_key())
        
        await client.send_message(event.chat_id, 'Ваш ключ, который необходимо отправить студентам:')
        key = generate()
        keyboard = [
            [  
                Button.inline("Назад", 'd')
            ]
        ]
        state[event.chat.id] = -1
#         name_op[event.chat.id] = 
        course00[key.get_key()] = event.message.message
        await client.send_message(event.chat_id, key.get_key(), buttons=keyboard)
    
    elif state[event.chat.id] == 2:
        if event.message.message not in course:
            state[event.chat.id] = 4
            keyboard = [
                [  
                    Button.inline("Попробовать еще раз", '2'),
                    Button.inline("Назад", '-1')
                ]
            ]
            await client.send_message(event.chat_id, 'Введен неверный ключ', buttons=keyboard)
        else:
            state[event.chat.id] = 5
            cur_op_in_teacher[event.chat.id] = course[event.message.message]
            await client.send_message(event.chat_id, f'Вы вошли в <{cur_op_in_teacher[event.chat.id]}>. \nВведите Ваше имя.\nЕсли хотите вернуться назад -- введите <Назад>')
            
    elif state[event.chat.id] == 5:
        if event.message.message == 'Назад' or event.message.message == 'назад':
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
        else:
            name[event.chat.id] = event.message.message
            state[event.chat.id] = 6
            await client.send_message(event.chat_id, f'Введите Вашу фамилию')
    
    elif state[event.chat.id] == 6:
        surname[event.chat.id] = event.message.message
        state[event.chat.id] = 7
        await client.send_message(event.chat_id, f'Введите Ваше отчество. При отсутствии напишите <Нет>')
        
    elif state[event.chat.id] == 7:
        flag = 1
        if event.message.message == 'Нет':
            last_name[event.chat.id] = ''
        else:
            last_name[event.chat.id] = event.message.message
        try:
            url_id = 'https://ruz.hse.ru/api/search?term=' + surname[event.chat.id] + '%' + name[event.chat.id] + '%' + last_name[event.chat.id]
            soup_id = requests.get(url_id).json()
            print(url_id)
            lect_id = []
            for mini_id in soup_id:
#                 if mini_id['type'] == 'lecturer':
                lect_id.append(mini_id['id'])
            disciplines = {}
        except:
            state[event.chat.id] = 8
            keyboard = [
                [  
                    Button.inline("Попробовать еще раз", '2'),
                    Button.inline("Назад", '-1')
                ]
            ]
            flag = 0
            await client.send_message(event.chat_id, 'Данного преподавателя не сущетствует, либо он не ведет никаких курсов', buttons=keyboard)
            
        if flag == 1:   
            for ids in lect_id:
                try:
                    url = 'https://ruz.hse.ru/api/schedule/lecturer/' + ids + '?start=2021.01.01&finish=2021.06.15'
                    data = requests.get(url).json()
                    for mini_data in data:
                        if mini_data['discipline'] not in list(disciplines.values()):
                            disciplines[mini_data['disciplineOid']] = mini_data['discipline']
                except:
                    pass
            keyboard = []
            azaz = disciplines
            for discipline_id in disciplines.keys():
                keyboard.append([Button.inline(disciplines[discipline_id], discipline_id)])
            keyboard.append([Button.inline('Назад', -1)])
            state[event.chat.id] = 9
            await client.send_message(event.chat_id, 'Выберите интересующий Вас курс', buttons=keyboard)
            
            
        
       
        
   
            
    elif state[event.chat.id] == 14:
        if (event.message.message, cur_discipline[event.chat.id]) in list(subject_name_to_id):
            keyboard = [
                [  
                    Button.inline("Ввести другую", '18'),
                    Button.inline("Назад", '10')
                ]
            ]
            state[event.chat.id] = 18
            await client.send_message(event.chat_id, 'Такая форма контроля уже существует', buttons=keyboard)
        else:
            cur_name[event.chat.id] = event.message.message
            subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])] = max_subject_id
            subject_id_to_name[max_subject_id] = (event.message.message, cur_discipline[event.chat.id])
            subject[max_subject_id] = []
            max_subject_id+=1
            keyboard = [
                [  
                    Button.inline("Назад", '19')
                ]
            ]
            state[event.chat.id]= 19 
            await client.send_message(event.chat_id, 'Успешно добавлено', buttons=keyboard)
            
    elif state[event.chat.id] == 15:
        if (event.message.message, cur_discipline[event.chat.id]) not in list(subject_name_to_id):
            keyboard = [
                [  
                    Button.inline("Ввести другую", '20'),
                    Button.inline("Назад", '10')
                ]
            ]
            state[event.chat.id] = 20
            await client.send_message(event.chat_id, 'Такой формы контроля не существует', buttons=keyboard)
        else:
            cur_name[event.chat.id] = event.message.message
            sub_id = subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            for i in range(len(subs) + 1):
                keyboard.append(Button.inline(str(i), i))
            state[event.chat.id] = 21
            await client.send_message(event.chat_id, 'Каким по счету поставить дедлайн?', buttons=keyboard)
        
    
        
    
        
    elif state[event.chat.id] == 24:
        status = event.message.message
        dedline[cur_ded[event.chat.id]] = (dedline[cur_ded[event.chat.id]][0], status, 0)
        state[event.chat.id] = 25
        
        await client.send_message(event.chat_id, 'Введите дату до которой ставить дедлайн в формате DD.MM.YYYY')
        
        
    elif state[event.chat.id] == 44:
        status = event.message.message
        dedline[cur_ded[event.chat.id]] = (dedline[cur_ded[event.chat.id]][0], status, 0)
        state[event.chat.id] = 45
        
        await client.send_message(event.chat_id, 'Введите дату до которой ставить дедлайн в формате DD.MM.YYYY')
        
    
    
    elif state[event.chat.id] == 25:
        status = event.message.message
        
        
        name_this = cur_name[event.chat.id]
        sub_id = subject_name_to_id[(name_this, cur_discipline[event.chat.id])]
        
        
        dedline[cur_ded[event.chat.id]] = (dedline[cur_ded[event.chat.id]][0], dedline[cur_ded[event.chat.id]][1], status)
        
        subject[sub_id].insert(cur_pos[event.chat.id], dedline[cur_ded[event.chat.id]])
        state[event.chat.id] = 26
        keyboard = [
                [  
                    Button.inline("Назад", '1')
                ]
            ]
        await client.send_message(event.chat_id, 'Успешно настроено', buttons=keyboard)
        
        
    elif state[event.chat.id] == 45:
        status = event.message.message
        
        
        name_this = cur_name[event.chat.id]
        sub_id = subject_name_to_id[(name_this, cur_discipline[event.chat.id])]
        
        
        dedline[cur_ded[event.chat.id]] = (dedline[cur_ded[event.chat.id]][0], dedline[cur_ded[event.chat.id]][1], status)
        
        subject[sub_id][cur_pos[event.chat.id]] =  dedline[cur_ded[event.chat.id]]
        state[event.chat.id] = 26
        keyboard = [
                [  
                    Button.inline("Назад", '1')
                ]
            ]
        await client.send_message(event.chat_id, 'Успешно настроено', buttons=keyboard)
        
    
    elif state[event.chat.id] == 16:
        if (event.message.message, cur_discipline[event.chat.id]) not in list(subject_name_to_id):
            keyboard = [
                [  
                    Button.inline("Ввести другую", '20'),
                    Button.inline("Назад", '10')
                ]
            ]
            state[event.chat.id] = 40
            await client.send_message(event.chat_id, 'Такой формы контроля не существует', buttons=keyboard)
        else:
            cur_name[event.chat.id] = event.message.message
            sub_id = subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            for i in range(len(subs)):
                keyboard.append(Button.inline(str(i), i))
            state[event.chat.id] = 41
            await client.send_message(event.chat_id, 'Каким по счету дедлайн Вы хотите изменить?', buttons=keyboard)
    
    elif state[event.chat.id] == 17:
        if (event.message.message, cur_discipline[event.chat.id]) not in list(subject_name_to_id):
            keyboard = [
                [  
                    Button.inline("Ввести другую", '20'),
                    Button.inline("Назад", '10')
                ]
            ]
            state[event.chat.id] = 60
            await client.send_message(event.chat_id, 'Такой формы контроля не существует', buttons=keyboard)
        else:
            cur_name[event.chat.id] = event.message.message
            sub_id = subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            keyboard.append(Button.inline('Удалить всю форму контроля', '0'))
            keyboard.append(Button.inline('Удалить дедлайн в форме контроля', '1'))
            keyboard.append(Button.inline('Назад', '2'))
            
            state[event.chat.id] = 61
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
    
    
        
    
@client.on(events.CallbackQuery)
async def callback(event):
    if state[event.chat.id] == 0:
        status = int(str(event.data).split("'")[1])
        row = [event.chat.id, status]
        users.loc[len(users)] = row
        state[event.chat.id] = status
        
        if state[event.chat.id] == 1:
            await client.send_message(event.chat, 'Пожалуйста, введите ключ, полученный от представителя университета')
            
        if state[event.chat.id] == 3:
            await client.send_message(event.chat, 'Выберите название для своего университета в системе')
            
        if state[event.chat.id] == 2:
            await client.send_message(event.chat, 'Пожалуйста, введите ключ, полученный от представителя университета')
            
            
        
    elif state[event.chat.id] == -1:
        keyboard = [
            [  
                Button.inline("Студент", '1'), 
                Button.inline("Преподаватель", '2')
            ],
            [
                Button.inline("Представитель нового университета", '3')
            ]
        ]
        state[event.chat.id] = 0
        await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
    
    elif state[event.chat.id] == 4:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            state[event.chat.id] = status
            await client.send_message(event.chat, 'Пожалуйста, введите ключ, полученный от представителя университета')
        else:
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
        
    elif state[event.chat.id] == 9:
        status = int(str(event.data).split("'")[1])
        if status == -1:
            state[event.chat.id] = 5
            await client.send_message(event.chat_id, f'Вы вошли в <{cur_op_in_teacher[event.chat.id]}>. \nВведите Ваше имя.\nЕсли хотите вернуться назад -- введите <Назад>')
        else:
            state[event.chat.id] = 10
            cur_discipline[event.chat.id] = status
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
            
    elif state[event.chat.id] == 200: 
        status = int(str(event.data).split("'")[1])
        if status == 2:
            state[event.chat.id] = status
            await client.send_message(event.chat, 'Пожалуйста, введите ключ, полученный от представителя университета')
        else:
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
        
    elif state[event.chat.id] == 19:
        state[event.chat.id] = 10
        keyboard = [
                [  
                    Button.inline("Добавить форму контроля", '11'), 
                    Button.inline("Добавить дедлайн внутри формы контроля", '12')
                ],
                [
                    Button.inline("Изменить форму контроля", '13'),
                    Button.inline("Удалить форму контроля", '14')
                ],
                    [
                        Button.inline("Назад", '102')
                    ]
            ]
        await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
    
    
            
            
    elif state[event.chat.id] == 61:
        status = int(str(event.data).split("'")[1])
        if status == 0:
            state[event.chat.id] = 19
            keyboard = []
            keyboard.append(Button.inline('Назад', 0))
            sub_id = subject_name_to_id[(cur_name[event.chat.id], cur_discipline[event.chat.id])]
            del subject[sub_id]
            del subject_name_to_id[(cur_name[event.chat.id], cur_discipline[event.chat.id])]
            await client.send_message(event.chat_id, 'Успешно удалено', buttons=keyboard)
        if status == 1:
            sub_id = subject_name_to_id[(cur_name[event.chat.id], cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            for i in range(len(subs)):
                keyboard.append(Button.inline(str(i), i))
            state[event.chat.id] = 64
            await client.send_message(event.chat_id, 'Каким по счету дедлайн Вы хотите удалить?', buttons=keyboard)
            
        if status == 2:
            state[event.chat.id] = 17
            
            
            
            cur_discipline[event.chat.id] = status
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)    
    
        
    elif state[event.chat.id] == 10:
        status = int(str(event.data).split("'")[1])
        if status == 102:
            url_id = 'https://ruz.hse.ru/api/search?term=' + surname[event.chat.id] + '%' + name[event.chat.id] + '%' + last_name[event.chat.id]
            soup_id = requests.get(url_id).json()
            lect_id = []
            for mini_id in soup_id:
                if mini_id['type'] == 'lecturer':
                    lect_id.append(mini_id['id'])

            disciplines = {}
            for ids in lect_id:
                url = 'https://ruz.hse.ru/api/schedule/lecturer/' + ids + '?start=2021.01.01&finish=2021.06.15'
                data = requests.get(url).json()
                for mini_data in data:
                    if mini_data['discipline'] not in list(disciplines.values()):
                        disciplines[mini_data['disciplineOid']] = mini_data['discipline']
            keyboard = []
            for discipline_id in disciplines.keys():
                keyboard.append([Button.inline(disciplines[discipline_id], discipline_id)])
            keyboard.append([Button.inline('Назад', -1)])
            state[event.chat.id] = 9
            await client.send_message(event.chat_id, 'Выберите интересующий Вас курс', buttons=keyboard)
            
        if status == 11:
            state[event.chat.id] = 14
            await client.send_message(event.chat_id, 'Введите название новой формы контроля')
            
        if status == 12:
            state[event.chat.id] = 15
            await client.send_message(event.chat_id, 'Введите название формы контроля')
            
        if status == 13:
            state[event.chat.id] = 16
            await client.send_message(event.chat_id, 'Введите название формы контроля')
        
        if status == 14:
            state[event.chat.id] = 17
            await client.send_message(event.chat_id, 'Введите название формы контроля')
    
    elif state[event.chat.id] == 18:
        status = int(str(event.data).split("'")[1])
        if status == 18:
            state[event.chat.id] = 14
            await client.send_message(event.chat_id, 'Введите название новой формы контроля')
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
        
    elif state[event.chat.id] == 20:
        status = int(str(event.data).split("'")[1])
        if status == 20:
            state[event.chat.id] = 15
            await client.send_message(event.chat_id, 'Введите название формы контроля')
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
            
            
            
    elif state[event.chat.id] == 60:
        status = int(str(event.data).split("'")[1])
        if status == 20:
            state[event.chat.id] = 17
            await client.send_message(event.chat_id, 'Введите название формы контроля')
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
        
    
    
    elif state[event.chat.id] == 8:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            state[event.chat.id] = 5
            await client.send_message(event.chat_id, 'Введите ваше имя')
        else:
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
            
    elif state[event.chat.id] == 208:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            state[event.chat.id] = 201
            await client.send_message(event.chat_id, 'Введите ваше имя')
        else:
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
            
    elif state[event.chat.id] == 23:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            cur_name[event.chat.id] = event.message.message
            sub_id = subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            for i in range(len(subs) + 1):
                keyboard.append(Button.inline(str(i), i))
            state[event.chat.id] = 21
            await client.send_message(event.chat_id, 'Каким по счету поставить дедлайн?', buttons=keyboard)
            
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
            
            
    elif state[event.chat.id] == 43:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            cur_name[event.chat.id] = event.message.message
            sub_id = subject_name_to_id[(event.message.message, cur_discipline[event.chat.id])]
            subs = subject[sub_id]
            keyboard = []
            for i in range(len(subs) + 1):
                keyboard.append(Button.inline(str(i), i))
            state[event.chat.id] = 41
            await client.send_message(event.chat_id, 'Каким по счету поставить дедлайн?', buttons=keyboard)
            
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
        
        
        

    
    
    elif state[event.chat.id] == 26:
        state[event.chat.id] = 10
        keyboard = [
                [  
                    Button.inline("Добавить форму контроля", '11'), 
                    Button.inline("Добавить дедлайн внутри формы контроля", '12')
                ],
                [
                    Button.inline("Изменить форму контроля", '13'),
                    Button.inline("Удалить форму контроля", '14')
                ],
                    [
                        Button.inline("Назад", '102')
                    ]
            ]
        await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
        
    
        
    elif state[event.chat.id] == -10:
        status = int(str(event.data).split("'")[1])
        if status == 20:
            state[event.chat.id] = 16
            await client.send_message(event.chat_id, 'Введите название формы контроля')
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
            
    elif state[event.chat.id] == 21:
        try:
            status = int(str(event.data).split("'")[1])
            cur_pos[event.chat.id] = status
            
            
            cur_ded[event.chat.id] = (cur_discipline[event.chat.id], status)
            dedline[(cur_discipline[event.chat.id], status)] = (0, 0, 0)
            state[event.chat.id] = 22
            keyboard = []
            tpr_keyboard = []
            for i in range(5):
                tpr_keyboard.append(Button.inline(str(i+1), i+1))
            keyboard.append(tpr_keyboard)
            tpr_keyboard = []
            for i in range(5, 10):
                tpr_keyboard.append(Button.inline(str(i+1), i+1))
            keyboard.append(tpr_keyboard)
            
            
            await client.send_message(event.chat_id, 'Выберите сложность дедлайна от 1 до 10', buttons=keyboard)
        except:
            keyboard = [
                [  
                    Button.inline("Повторить попытку", '2'),
                    Button.inline("Назад", '1')
                ]
            ]
            state[event.chat.id] = 23
            await client.send_message(event.chat_id, 'Что-то введено не так.', buttons=keyboard)
            
            
            
    elif state[event.chat.id] == 22:
        status = int(str(event.data).split("'")[1])
        dedline[cur_ded[event.chat.id]] = (status, 0, 0)
        state[event.chat.id] = 24
        
        
        await client.send_message(event.chat_id, 'Введите дату от которой ставить дедлайн в формате DD.MM.YYYY')
            
        
        
    elif state[event.chat.id] == 40:
        status = int(str(event.data).split("'")[1])
        if status == 20:
            state[event.chat.id] = 16
            await client.send_message(event.chat_id, 'Введите название формы контроля')
        else:
            state[event.chat.id] = 10
            keyboard = [
                    [  
                        Button.inline("Добавить форму контроля", '11'), 
                        Button.inline("Добавить дедлайн внутри формы контроля", '12')
                    ],
                    [
                        Button.inline("Изменить форму контроля", '13'),
                        Button.inline("Удалить форму контроля", '14')
                    ],
                    [
                        Button.inline("Назад", '102')
                    ]
                ]
            await client.send_message(event.chat_id, 'Выберите действие', buttons=keyboard)
            
    elif state[event.chat.id] == 41:
        try:
            status = int(str(event.data).split("'")[1])
            cur_pos[event.chat.id] = status
            
            
            cur_ded[event.chat.id] = (cur_discipline[event.chat.id], status)
            dedline[(cur_discipline[event.chat.id], status)] = (0, 0, 0)
            state[event.chat.id] = 42
            keyboard = []
            tpr_keyboard = []
            for i in range(5):
                tpr_keyboard.append(Button.inline(str(i+1), i+1))
            keyboard.append(tpr_keyboard)
            tpr_keyboard = []
            for i in range(5, 10):
                tpr_keyboard.append(Button.inline(str(i+1), i+1))
            keyboard.append(tpr_keyboard)
            
            
            await client.send_message(event.chat_id, 'Выберите сложность дедлайна от 1 до 10', buttons=keyboard)
        except:
            keyboard = [
                [  
                    Button.inline("Повторить попытку", '2'),
                    Button.inline("Назад", '1')
                ]
            ]
            state[event.chat.id] = 43
            await client.send_message(event.chat_id, 'Что-то введено не так.', buttons=keyboard)
        
    elif state[event.chat.id] == 42:
        status = int(str(event.data).split("'")[1])
        dedline[cur_ded[event.chat.id]] = (status, 0, 0)
        state[event.chat.id] = 44
        
        
        await client.send_message(event.chat_id, 'Введите дату от которой ставить дедлайн в формате DD.MM.YYYY')
        
        
    elif state[event.chat.id] == 64:
        status = int(str(event.data).split("'")[1])
        cur_pos[event.chat.id] = status

        del subject[subject_name_to_id[(cur_name[event.chat.id], cur_ded[event.chat.id][0])]][status]
        keyboard = []
        keyboard.append(Button.inline('Назад', 1))
        state[event.chat.id] = course00[key.get_key()]
        await client.send_message(event.chat_id, 'Успешно удалено', buttons=keyboard)
        
        
    elif state[event.chat.id] == 222:
        status = int(str(event.data).split("'")[1])
        if status == 2:
            await client.send_message(event.chat_id, 'Введите Ваш gmail')
            state[event.chat.id] = 223
        else:
            keyboard = [
                [  
                    Button.inline("Студент", '1'), 
                    Button.inline("Преподаватель", '2')
                ],
                [
                    Button.inline("Представитель нового университета", '3')
                ]
            ]
            state[event.chat.id] = 0
            await client.send_message(event.chat_id, "Кто вы:", buttons=keyboard)
        
        

In [10]:
cur_stud_surname

{341821214: 'Котов'}

In [13]:
from scipy import stats
stats.ttest_ind([1,2,3], [2,3,1])

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [33]:
stats.ttest_ind([10,6,5,7,3,8,7,9,10,7,6,7,8,5,6,7,6,7,9,6], [10,6,5,9,5,8,10,9,10,7,10,9,8,5,6,7,8,7,9,10])

Ttest_indResult(statistic=-2.210365192839022, pvalue=0.03317480947432919)

In [25]:
stats.ttest_ind([10,6,5,7,3,8,7,9,10,7,6,7,8,5,6,7,6,7,9,6], [9,6,5,7,3,8,7,9,10,7,6,7,8,5,6,7,6,7,9,6])

Ttest_indResult(statistic=0.0934646039092232, pvalue=0.9260255326128035)

In [26]:
np.array([10,6,5,7,3,8,7,9,10,7,6,7,8,5,6,7,6,7,9,6]).mean()

6.95

In [28]:
np.array([12,6,5,9,5,8,7,9,10,7,10,9,8,5,6,7,8,7,9,10]).mean()

7.85

In [28]:
dedlines_by_lesson = {}
tpr_lst = list(subject_id_to_name.keys())
for i in range(len(subject_id_to_name)):
    if subject_id_to_name[i][1] in dedlines_by_lesson:
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])
    else:
        dedlines_by_lesson[subject_id_to_name[i][1]] = []
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])


get_dead = {}
ccc = 0

m = GEKKO()
X = []
W = []
D_s = []
D_e = []
Students = []
# x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]


# m.Equation(x1+x2+x3+x4<=25)




for i in range(len(subject)):
    qq = list(subject.keys())[i]
    idd = subject_id_to_name[qq][1]
    get_dead[tpr_lst[i]] = []
    for j in range(len(subject[qq])):
        day_start = subject[qq][j][1].split('.')[0]
        month_start = subject[qq][j][1].split('.')[1]
        year_start = subject[qq][j][1].split('.')[2]

        day_end = subject[qq][j][2].split('.')[0]
        month_end = subject[qq][j][2].split('.')[1]
        year_end = subject[qq][j][2].split('.')[2]

        date_start = date(int(year_start),int(month_start),int(day_start))
        date_end = date(int(year_end),int(month_end),int(day_end))

        tpr_date = date(2019,1,1)
        date_start = (date_start - tpr_date).days
        date_end = (date_end - tpr_date).days
        D_s.append(date_start)
        D_e.append(date_end)
        get_dead[tpr_lst[i]].append(ccc)
        ccc+=1
        X.append(m.Var(lb=date_start, ub=date_end))
        W.append(subject[i][j][0])
        if j != 0:
            m.Equation(X[-1] > X[-2] + 0.4)


student_deadlines = []
for i in range(len(student_subjects)):
    tt = []
    for j in range(len(student_subjects[i])):
        try:
            tt = dedlines_by_lesson[student_subjects[i][j]]
        except:
            pass
    student_deadlines.append(tt)

cnt = 0
cnt2=0
expr = []

P = 0
for stud in range(len(student_deadlines)):
    lessons = []
#     if cnt > 15:
#         break
    for i in range(len(student_deadlines[stud])):
        lessons = lessons + get_dead[student_deadlines[stud][i]]
#         print(get_dead[student_deadlines[stud][i]])
#     print(stud)
    for i in range(len(lessons)):
        print(stud)
        for j in range(i+1, len(lessons)):
#             P += (W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2
            expr.append((W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2)
#             print((W[lessons[i]] + W[lessons[j]]))
#             print(np.abs(1 + X[lessons[i]] - X[lessons[j]]))
            cnt2+=1
    if len(lessons) > 0:
        cnt+=1

for i in range(len(expr)):
    m.Obj(expr[i])

m.solve()
# try: 
url_id = 'https://ruz.hse.ru/api/search?term=' + cur_stud_surname[341821214] + '%' + cur_stud_name[341821214] + '%' + cur_stud_last_name[341821214]
soup_id = requests.get(url_id).json()
ids_me = []
for i in range(len(soup_id)):
    if soup_id[i]['type'] == 'student':
        ids_me.append(soup_id[i]['id'])
url = 'https://ruz.hse.ru/api/schedule/student/' + ids_me[0] + '?start=2021.01.01&finish=2021.06.11'
data = requests.get(url).json()
my_disciplines = {}
for mini_data in data:
    if mini_data['discipline'] not in list(my_disciplines.values()):
        my_disciplines[mini_data['disciplineOid']] = mini_data['discipline']

idx = []
for less in range(len(subject_id_to_name)):
    if list(subject_id_to_name.values())[less][1] in list(my_disciplines.keys()):
        idx.append(less)

kk = list(get_dead.keys())
vv = list(get_dead.values())

ans = []
ppp = 0
print(idx)
print(kk)
print(my_disciplines)
for i in range(len(kk)):

    ee = 1
    for j in range(len(get_dead[kk[i]])):
        if kk[i] in idx:
            ans.append((str(tpr_date + datetime.timedelta(days=X[ppp][0])), my_disciplines[subject_id_to_name[kk[i]][1]], subject_id_to_name[kk[i]][0] + '-' + str(ee)))
        ppp+=1
        ee+=1
print(ans)
sheet = client2.open('Таблица').sheet1
pp_ = pprint.PrettyPrinter()
results = sheet.get_all_records()
ddf = pd.DataFrame(ans, columns=['0', '1', '2']).sort_values(by='0').values
for i in range(len(ans)):
    sheet.append_row(list(ddf[i]))

keyboard = [
    [  
        Button.inline("Хочу синхронизовать расписание дедлайнов с календарем", '2'),
        Button.inline("Назад", '-1')
    ]
]

# anses[event.chat_id] = ans
# await client.send_message(event.chat_id, 'Ваше расписание https://docs.google.com/spreadsheets/d/1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU/edit#gid=0', buttons=keyboard)

# state[event.chat.id] = 222


# except:
#     state[event.chat.id] = 208
#     keyboard = [
#         [  
#             Button.inline("Попробовать еще раз", '2'),
#             Button.inline("Назад", '-1')
#         ]
#     ]

11
11
11
11
48
48
48
48
apm 83.220.236.245_gk_model15 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            6
   Intermediates:            0
   Connections  :            0
   Equations    :           14
   Residuals    :           14
 
 Number of state variables:              6
 Number of total equations: -            2
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :              2
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

*******************************************************************

IndexError: list index out of range

In [19]:
calendar_id

'e6j3rcjmabkdbc50sfonbfa1ac@group.calendar.google.com'

In [8]:
await client.run_until_disconnected()

d
Высшая Школа Экономики
44b2f1-ae686ec-e92c-4d8-e2079
Никита
Орлов
Андреевич
https://ruz.hse.ru/api/search?term=Орлов%Никита%Андреевич
никита
орлов
андреевич
https://ruz.hse.ru/api/search?term=орлов%никита%андреевич
никита
орлов
андреевич
https://ruz.hse.ru/api/search?term=орлов%никита%андреевич
антон
конушин
сергеевич
https://ruz.hse.ru/api/search?term=конушин%антон%сергеевич
Контест
Контест
01.01.2021
15.01.2021
Контест
05.01.2021
30.01.2021
Контест
17.01.2021
12.02.2021
Контест
25.01.2021
25.02.2021
Контест
10.02.2021
10.03.2021
Контест
18.02.2021
18.03.2021
Контест
25.02.2021
25.03.2021
Экзамен
Экзамен
15.03.2021
25.03.2021
Коллоквиум
Коллоквиум
10.02.2021
25.02.2021
Назад
44b2f1-ae686ec-e92c-4d8-e2079
Кирилл
Струминский
Алексеевич
https://ruz.hse.ru/api/search?term=Струминский%Кирилл%Алексеевич
Артем
Бабенко
Валерьевич
https://ruz.hse.ru/api/search?term=Бабенко%Артем%Валерьевич
никита
орлов
андреевич
https://ruz.hse.ru/api/search?term=орлов%никита%андреевич
артем
бабенко
валерьев

ERROR:telethon.client.updates:Unhandled exception on handler
Traceback (most recent call last):
  File "/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/telethon/client/updates.py", line 454, in _dispatch_update
    await callback(event)
  File "<ipython-input-7-34ad3339b0f7>", line 306, in handler
    state[event.chat.id] = -1
AttributeError: 'dict' object has no attribute 'chat'
ERROR:telethon.network.mtprotosender:Automatic reconnection failed 5 time(s)
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=ConnectionError('Connection to Telegram failed 5 time(s)')>
ConnectionError: Connection to Telegram failed 5 time(s)


ConnectionError: Connection to Telegram failed 5 time(s)

In [ ]:
Зимовно́в Андрей Вадимович

In [ ]:
https://www.google.com/calendar/event?eid=MzAzaWh0OXNzMDZwcHY2bDZxM2RrNHIyYm8gY29jMjNlM2RrMDA5bWpkZ2J2dGFmaTczZ2NAZw

In [16]:
calendars_result = service.calendarList().list().execute()

calendars = calendars_result.get('items', [])

calendar_id = calendars[-2]['id']

In [14]:
calendars[-2]

{'kind': 'calendar#calendarListEntry',
 'etag': '"1620725266035000"',
 'id': 'e6j3rcjmabkdbc50sfonbfa1ac@group.calendar.google.com',
 'summary': 'Календарь расписания дедлайнов',
 'timeZone': 'Europe/Moscow',
 'colorId': '11',
 'backgroundColor': '#fbe983',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [],
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [15]:
calendars[-]

{'kind': 'calendar#calendarListEntry',
 'etag': '"1620737956191000"',
 'id': 'egokoto31@gmail.com',
 'summary': 'egokoto31@gmail.com',
 'timeZone': 'Europe/Moscow',
 'colorId': '14',
 'backgroundColor': '#9fe1e7',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'notificationSettings': {'notifications': [{'type': 'eventCreation',
    'method': 'email'},
   {'type': 'eventChange', 'method': 'email'},
   {'type': 'eventCancellation', 'method': 'email'},
   {'type': 'eventResponse', 'method': 'email'}]},
 'primary': True,
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [20]:
event.get('htmlLink')

'https://www.google.com/calendar/event?eid=ampsNjI0dTZtamdwOG9iZGh1NGs1MTQwZDQgZWdva290bzMxQG0'

In [105]:
class Genetic(object):

    def __init__(self, f, pop_size = 100, n_variables = 2):
        self.f = f
        self.minim = -100
        self.maxim = 100
        self.pop_size = pop_size
        self.n_variables = n_variables
        self.population = self.initializePopulation()
        self.evaluatePopulation()

    def initializePopulation(self):
        return [np.random.randint(self.minim, self.maxim, size=(self.n_variables)) 
                           for i in range(self.pop_size)]

    def evaluatePopulation(self):
        return [self.f(i[0], i[1]) for i in self.population]
        #return [(i[0]-4)**2 + i[1]**2 for i in self.population]

    def nextGen(self):
        results = self.evaluatePopulation()
        children = [self.population[np.argmin(results)]]

        while len(children) < self.pop_size:
            # Tournament selection
            randA, randB = np.random.randint(0, self.pop_size), \
                           np.random.randint(0, self.pop_size)
            if results[randA] < results[randB]: p1 = self.population[randA]
            else: p1 = self.population[randB]

            randA, randB = np.random.randint(0, self.pop_size), \
                           np.random.randint(0, self.pop_size)  
            if results[randA] < results[randB]: p2 = self.population[randA]
            else: p2 = self.population[randB]   

            signs = []
            for i in zip(p1, p2):
                if i[0] < 0 and i[1] < 0: signs.append(-1)
                elif i[0] >= 0 and i[1] >= 0: signs.append(1)
                else: signs.append(np.random.choice([-1,1]))

            # Convert values to binary
            p1 = [format(abs(i), '010b') for i in p1]
            p2 = [format(abs(i), '010b') for i in p2]

            # Recombination
            child = []
            for i, j in zip(p1, p2):
                for k, l in zip(i, j):
                    if k == l: child.append(k)
                    else: child.append(str(np.random.randint(min(k, l), 
                                                             max(k,l))))

            child = ''.join(child)
            g1 = child[0:len(child)//2] 
            g2 = child[len(child)//2:len(child)]
            children.append(np.asarray([signs[0]*int(g1, 2), 
                                        signs[1]*int(g2, 2)]))
        self.population = children

    def run(self):
        ix = 0
        while ix < 1000:
            ix += 1
            print(self.population[0])
            self.nextGen()
        return self.population[0]

In [108]:
f = lambda x, y: (-x**2 + y**3 + y**4 + x**3 - 5 + x - 10*y**2 +2*x**4)
gen = Genetic(f)
minim = gen.run()
print('Minimum found      X =', minim[0], ', Y =', minim[1])

[-97  27]
[8 5]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]
[-1 -2]


In [84]:
cur_stud_last_name

{341821214: 'Александрович'}

In [91]:
day_end

'30'

In [27]:
surname

{341821214: 'Орлов'}

In [28]:
url_id = 'https://ruz.hse.ru/api/search?term=' + surname[341821214] + '%' + name[341821214] + '%' + last_name[341821214]
soup_id = requests.get(url_id).json()
lect_id = []
for mini_id in soup_id:
#                 if mini_id['type'] == 'lecturer':
    lect_id.append(mini_id['id'])
print(lect_id)
disciplines = {}
for ids in lect_id:
    url = 'https://ruz.hse.ru/api/schedule/lecturer/' + ids + '?start=2021.01.01&finish=2021.06.15'
    data = requests.get(url).json()
    for mini_data in data:
        if mini_data['discipline'] not in list(disciplines.values()):
            disciplines[mini_data['disciplineOid']] = mini_data['discipline']

['47439', '4403959372', '246008']


In [8]:
await client.run_until_disconnected()

d
в
a24c2d2-f0bed-86ea3061b-8041621-0fb9b6
Никита
орлов
андреевич
https://ruz.hse.ru/api/search?term=орлов%Никита%андреевич
Никита
орлов
андреевич
https://ruz.hse.ru/api/search?term=орлов%Никита%андреевич
Домашняя работа
Домашняя работа
15.01.2021
28.01.2021
Домашняя работа
04.02.2021
20.02.2021
Назад
db810a-6878d3b5-8fa72a-6efc-1c1aaccc5
Егор
Котов
Александрович
11
11
54
54
68
68
apm 83.220.236.245_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            3
   Intermediates:            0
   Connections  :            0
   Equations    :            4
   Residuals    :            4
 
 Number of state variables:              3
 Number of total equations: -            1
 Number of slac

CancelledError: 

In [45]:
ans = [('кр-1', datetime.date(2021, 4, 1), 'Машинное обучение для больших данных (рус)'), ('кр-2', datetime.date(2021, 4, 25), 'Машинное обучение для больших данных (рус)')]

In [21]:
def f(x):
    x = x.split(' ')
    l = []
    for i in range(len(x)):
        if x in companies:
            l.append(x)
    return l

df['new_column'] = df['news'].apply(f)


IndentationError: expected an indented block (<ipython-input-21-c1ea60ffb4c4>, line 5)

In [72]:
sheet.append_row(list(pd.DataFrame(ans, columns=['0', '1', '2']).sort_values(by='2').values[0]))

{'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
 'tableRange': "'Лист1'!A1:C6",
 'updates': {'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
  'updatedRange': "'Лист1'!A7:C7",
  'updatedRows': 1,
  'updatedColumns': 3,
  'updatedCells': 3}}

In [71]:
list(pd.DataFrame(ans, columns=['0', '1', '2']).sort_values(by='2').values[0])

['кр-1', '2021-04-01', 'Машинное обучение для больших данных (рус)']

In [67]:
pd.DataFrame(ans, columns=['0', '1', '2']).sort_values(by='2').values

'кр-1'

In [ ]:
Обучение с подкреплением Бабенко Артем Валерьевич
Машинное обучение для больших данных (рус) Орлов Никита Андреевич
Научно-исследовательский семинар Струминский Кирилл Алексеевич
Компьютерное зрение Конушин Антон Сергеевич

In [42]:
import numpy as np
from scipy.optimize import fmin
import math

def f(x):
    exp = (math.pow(x[0], 2) + math.pow(x[1], 2)) * -1
    return math.exp(exp) * math.cos(x[0] * x[1]) * math.sin(x[0] * x[1])

fmin(f,np.array([0,0]))

Optimization terminated successfully.
         Current function value: -0.161198
         Iterations: 60
         Function evaluations: 113


array([ 0.62665701, -0.62663095])

In [48]:
fmin(f,np.array([0,0,0,0]))

array([ 2.33508288e+48, -4.03363155e+48,  1.83832662e+48,  8.34260903e+46])

In [47]:
def f(x):
    P = 0
    for stud in range(len(student_deadlines)):
        lessons = []
        for i in range(len(student_deadlines[stud])):
            lessons = lessons + get_dead[student_deadlines[stud][i]]

        for i in range(len(lessons)):
            for j in range(i+1, len(lessons)):
                P += (W[lessons[i]] + W[lessons[j]]) / np.abs(1e-2 + x[lessons[i]] - x[lessons[j]])
    return P

In [43]:
P

(((((((((((((((((((((0+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(12/abs(((0.01+v3)-v4))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(12/abs(((0.01+v3)-v4))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(12/abs(((0.01+v3)-v4))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(12/abs(((0.01+v3)-v4))))+(12/abs(((0.01+v3)-v4))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))+(11/abs(((0.01+v1)-v2))))

In [380]:
subject_name_to_id

{('Домашняя работа', 61051): 0, ('Контрольная работа', 61051): 1}

In [382]:
subject_id_to_name

{0: ('Домашняя работа', 61051),
 1: ('Контрольная работа', 61051),
 2: ('л', 61051)}

In [383]:
dedlines_by_lesson = {}
tpr_lst = list(subject_id_to_name.keys())
for i in range(len(subject_id_to_name)):
    if subject_id_to_name[i][1] in dedlines_by_lesson:
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])
    else:
        dedlines_by_lesson[subject_id_to_name[i][1]] = []
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])
        
        
get_dead = {}
ccc = 0

m = GEKKO()
X = []
W = []
D_s = []
D_e = []
Students = []
# x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]


# m.Equation(x1+x2+x3+x4<=25)




for i in range(len(subject)):
    qq = list(subject.keys())[i]
    idd = subject_id_to_name[qq][1]
    get_dead[tpr_lst[i]] = []
    for j in range(len(subject[qq])):
        day_start = subject[qq][j][1].split('.')[0]
        month_start = subject[qq][j][1].split('.')[1]
        year_start = subject[qq][j][1].split('.')[2]
        
        day_end = subject[qq][j][2].split('.')[0]
        month_end = subject[qq][j][2].split('.')[1]
        year_end = subject[qq][j][2].split('.')[2]
        
        date_start = date(int(year_start),int(month_start),int(day_start))
        if()
        date_end = date(int(year_end),int(month_end),int(day_end))
        
        tpr_date = date(2019,1,1)
        date_start = (date_start - tpr_date).days
        date_end = (date_end - tpr_date).days
        D_s.append(date_start)
        D_e.append(date_end)
        get_dead[tpr_lst[i]].append(ccc)
        ccc+=1
        X.append(m.Var(lb=date_start, ub=date_end))
        W.append(subject[i][j][0])
        if j != 0:
            m.Equation(X[-1] > X[-2] + 0.4)
            
            
student_deadlines = []
for i in range(len(student_subjects)):
    tt = []
    for j in range(len(student_subjects[i])):
        try:
            tt = dedlines_by_lesson[student_subjects[i][j]]
        except:
            pass
    student_deadlines.append(tt)
    
cnt = 0
cnt2=0
expr = []

P = 0
for stud in range(len(student_deadlines)):
    lessons = []
#     if cnt > 15:
#         break
    for i in range(len(student_deadlines[stud])):
        lessons = lessons + get_dead[student_deadlines[stud][i]]
#         print(get_dead[student_deadlines[stud][i]])
#     print(stud)
    for i in range(len(lessons)):
        print(stud)
        for j in range(i+1, len(lessons)):
#             P += (W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2
            expr.append((W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2)
#             print((W[lessons[i]] + W[lessons[j]]))
#             print(np.abs(1 + X[lessons[i]] - X[lessons[j]]))
            cnt2+=1
    if len(lessons) > 0:
        cnt+=1
        
for i in range(len(expr)):
    m.Obj(expr[i])
    
m.solve()

{61051: [0, 1]}

In [168]:
dedlines_by_lesson = {}
tpr_lst = list(subject_id_to_name.keys())
for i in range(len(subject_id_to_name)):
    if subject_id_to_name[i][1] in dedlines_by_lesson:
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])
    else:
        dedlines_by_lesson[subject_id_to_name[i][1]] = []
        dedlines_by_lesson[subject_id_to_name[i][1]].append(tpr_lst[i])

In [260]:
get_dead = {}
ccc = 0

m = GEKKO()
X = []
W = []
D_s = []
D_e = []
Students = []
# x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]


# m.Equation(x1+x2+x3+x4<=25)




for i in range(len(subject)):
    qq = list(subject.keys())[i]
    idd = subject_id_to_name[qq][1]
    get_dead[tpr_lst[i]] = []
    for j in range(len(subject[qq])):
        day_start = subject[qq][j][1].split('.')[0]
        month_start = subject[qq][j][1].split('.')[1]
        year_start = subject[qq][j][1].split('.')[2]
        
        day_end = subject[qq][j][2].split('.')[0]
        month_end = subject[qq][j][2].split('.')[1]
        year_end = subject[qq][j][2].split('.')[2]
        
        date_start = date(int(year_start),int(month_start),int(day_start))
        date_end = date(int(year_end),int(month_end),int(day_end))
        
        tpr_date = date(2019,1,1)
        date_start = (date_start - tpr_date).days
        date_end = (date_end - tpr_date).days
        D_s.append(date_start)
        D_e.append(date_end)
        get_dead[tpr_lst[i]].append(ccc)
        ccc+=1
        X.append(m.Var(lb=date_start, ub=date_end))
        W.append(subject[i][j][0])
        if j != 0:
            m.Equation(X[-1] > X[-2] + 0.4)

In [227]:
X

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [133]:
subject[qq][j]

(3, 'й', 'й')

In [26]:
fkn_ids = list(pd.read_csv('fkn2.csv', sep=';')['id'])
student_subjects = []
student_number = 0
for i in tqdm(range(len(fkn_ids))):
    url = 'https://ruz.hse.ru/api/schedule/student/' + str(fkn_ids[i]) + '?start=2021.01.01&finish=2021.06.15'
    ss = requests.get(url).json()
    time.sleep(1)
    sss = []
    for i in range(len(ss)):
        sss.append(ss[i]['disciplineOid'])
    sss = list(set(sss))
    student_subjects.append(sss)


 19%|█▉        | 465/2457 [10:25<44:39,  1.35s/it]  


KeyboardInterrupt: 

In [19]:
student_subjects = pd.read_csv('fkn.csv')

In [22]:
student_subjects = student_subjects.values.tolist()

In [23]:
len(student_subjects)

2457

In [11]:
pd.DataFrame(student_subjects).to_csv('fkn.csv')

In [229]:
student_deadlines = []
for i in range(len(student_subjects)):
    tt = []
    for j in range(len(student_subjects[i])):
        try:
            tt = dedlines_by_lesson[student_subjects[i][j]]
        except:
            pass
    student_deadlines.append(tt)

In [177]:
student_deadlines[stud][2]

2

In [180]:
get_dead

{0: [0, 1, 2, 3, 4, 5, 6, 7], 1: [8, 9]}

In [179]:
get_dead[student_deadlines[stud][i]]

KeyError: 2

In [186]:
dedlines_by_lesson[61051] = [0,1]

In [385]:
lessons

[]

In [183]:
student_deadlines

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []

In [386]:
get_dead

{0: [0, 1, 2, 3, 4, 5, 6, 7], 1: [8, 9]}

In [249]:
cnt = 0
cnt2=0

In [250]:
expr = []

In [411]:
cnt = 0
cnt2=0
expr = []

P = 0
for stud in range(len(student_deadlines)):
    lessons = []
#     if cnt > 15:
#         break
    for i in range(len(student_deadlines[stud])):
        lessons = lessons + get_dead[student_deadlines[stud][i]]
#         print(get_dead[student_deadlines[stud][i]])
#     print(stud)
    for i in range(len(lessons)):
        print(stud)
        for j in range(i+1, len(lessons)):
#             P += (W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2
            expr.append((W[lessons[i]] + W[lessons[j]]) / (1e-2 + X[lessons[i]] - X[lessons[j]])**2)
#             print((W[lessons[i]] + W[lessons[j]]))
#             print(np.abs(1 + X[lessons[i]] - X[lessons[j]]))
            cnt2+=1
    if len(lessons) > 0:
        cnt+=1

48
48
48
48
48
48
48
48
48
48
60
60
60
60
60
60
60
60
60
60
68
68
68
68
68
68
68
68
68
68
82
82
82
82
82
82
82
82
82
82
108
108
108
108
108
108
108
108
108
108
129
129
129
129
129
129
129
129
129
129
217
217
217
217
217
217
217
217
217
217
242
242
242
242
242
242
242
242
242
242
269
269
269
269
269
269
269
269
269
269
318
318
318
318
318
318
318
318
318
318
325
325
325
325
325
325
325
325
325
325
329
329
329
329
329
329
329
329
329
329
331
331
331
331
331
331
331
331
331
331
351
351
351
351
351
351
351
351
351
351
358
358
358
358
358
358
358
358
358
358
369
369
369
369
369
369
369
369
369
369
376
376
376
376
376
376
376
376
376
376
379
379
379
379
379
379
379
379
379
379
384
384
384
384
384
384
384
384
384
384
414
414
414
414
414
414
414
414
414
414
438
438
438
438
438
438
438
438
438
438
448
448
448
448
448
448
448
448
448
448
479
479
479
479
479
479
479
479
479
479
486
486
486
486
486
486
486
486
486
486
487
487
487
487
487
487
487
487
487
487
488
488
488
488
488
488
488
488
488
488


In [245]:
expr

[(12/((((0.01+v1)-v2))^(2))),
 (8/((((0.01+v1)-v3))^(2))),
 (10/((((0.01+v1)-v4))^(2))),
 (13/((((0.01+v1)-v5))^(2))),
 (14/((((0.01+v1)-v6))^(2))),
 (10/((((0.01+v1)-v7))^(2))),
 (13/((((0.01+v1)-v8))^(2))),
 (14/((((0.01+v1)-v9))^(2))),
 (15/((((0.01+v1)-v10))^(2))),
 (10/((((0.01+v2)-v3))^(2))),
 (12/((((0.01+v2)-v4))^(2))),
 (15/((((0.01+v2)-v5))^(2))),
 (16/((((0.01+v2)-v6))^(2))),
 (12/((((0.01+v2)-v7))^(2))),
 (15/((((0.01+v2)-v8))^(2))),
 (16/((((0.01+v2)-v9))^(2))),
 (17/((((0.01+v2)-v10))^(2))),
 (8/((((0.01+v3)-v4))^(2))),
 (11/((((0.01+v3)-v5))^(2))),
 (12/((((0.01+v3)-v6))^(2))),
 (8/((((0.01+v3)-v7))^(2))),
 (11/((((0.01+v3)-v8))^(2))),
 (12/((((0.01+v3)-v9))^(2))),
 (13/((((0.01+v3)-v10))^(2))),
 (13/((((0.01+v4)-v5))^(2))),
 (14/((((0.01+v4)-v6))^(2))),
 (10/((((0.01+v4)-v7))^(2))),
 (13/((((0.01+v4)-v8))^(2))),
 (14/((((0.01+v4)-v9))^(2))),
 (15/((((0.01+v4)-v10))^(2))),
 (17/((((0.01+v5)-v6))^(2))),
 (13/((((0.01+v5)-v7))^(2))),
 (16/((((0.01+v5)-v8))^(2))),
 (17/((((

In [31]:
X[0].value = 1
X[1].value = 5
X[2].value = 5

In [414]:
student_deadlines[48]

[0, 1]

In [262]:
# m.Obj(sum(expr))

# modified 1
for i in range(len(expr)):
    m.Obj(expr[i])
    
    

In [ ]:
m.Obj(m.sum(expr))

In [263]:
# m.Obj(P)

m.solve()

#Results
# print('')
# print('Results')
# print('x1: ' + str(X[0].value))
# print('x2: ' + str(X[1].value))
# print('x3: ' + str(X[2].value))
# print('x4: ' + str(x4.value))

apm 95.165.169.196_gk_model13 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           18
   Intermediates:            0
   Connections  :            0
   Equations    :         4688
   Residuals    :         4688
 
 Number of state variables:             18
 Number of total equations: -            8
 Number of slack variables: -            8
 ---------------------------------------
 Degrees of freedom       :              2
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program

In [387]:
get_dead

{0: [0, 1, 2, 3, 4, 5, 6, 7], 1: [8, 9]}

In [436]:
subject_id_to_name

{0: ('Домашняя работа', 61051), 1: ('Контрольная работа', 61051)}

In [389]:

# for i in range(len())

In [447]:
list(get_dead.keys())

[0, 1]

In [473]:
subject_id_to_name[0]

('Домашняя работа', 61051)

In [483]:
kk

[0, 1]

In [484]:
ans

[('Домашняя работа-1',
  datetime.date(2021, 1, 20),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-2',
  datetime.date(2021, 1, 28),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-3',
  datetime.date(2021, 2, 5),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-4',
  datetime.date(2021, 2, 19),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-5',
  datetime.date(2021, 2, 27),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-6',
  datetime.date(2021, 3, 7),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-7',
  datetime.date(2021, 3, 23),
  'Обучение с подкреплением (рус)'),
 ('Домашняя работа-8',
  datetime.date(2021, 3, 30),
  'Обучение с подкреплением (рус)'),
 ('Контрольная работа-1',
  datetime.date(2021, 2, 12),
  'Обучение с подкреплением (рус)'),
 ('Контрольная работа-2',
  datetime.date(2021, 3, 15),
  'Обучение с подкреплением (рус)')]

In [409]:
final = []

In [22]:
url_id = 'https://ruz.hse.ru/api/search?term=' + 'Котов' + '%' + 'Егор' + '%' + 'Александрович'
soup_id = requests.get(url_id).json()
ids_me = []
for i in range(len(soup_id)):
    if soup_id[i]['type'] == 'student':
        ids_me.append(soup_id[i]['id'])
url = 'https://ruz.hse.ru/api/schedule/student/' + ids_me[0] + '?start=2021.01.01&finish=2021.06.11'
data = requests.get(url).json()
my_disciplines = {}
for mini_data in data:
    if mini_data['discipline'] not in list(my_disciplines.values()):
        my_disciplines[mini_data['disciplineOid']] = mini_data['discipline']

In [23]:
my_disciplines

{61061: 'Машинное обучение для больших данных (рус)',
 58665: 'Научно-исследовательский семинар "Машинное обучение и приложения 2" (рус)',
 61051: 'Обучение с подкреплением (рус)',
 61046: 'Компьютерное зрение (рус)',
 60317: 'Соревновательный анализ данных (анг)'}

In [21]:
list(disciplines.values())

[]

In [259]:
for i in range(len(X)):
    print(tpr_date + datetime.timedelta(days=X[i][0]))

2021-01-20
2021-01-28
2021-02-05
2021-02-19
2021-02-26
2021-03-07
2021-03-23
2021-03-29
2021-02-12
2021-03-15


In [265]:
import gspread
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials2.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
#Now will can access our google sheets we call client.open on StartupName
sheet = client.open('Таблица').sheet1
pp = pprint.PrettyPrinter()
results = sheet.get_all_records()
results

sheet.append_row([1,2])

2021-01-20
2021-01-28
2021-02-05
2021-02-19
2021-02-27
2021-03-07
2021-03-23
2021-03-30
2021-02-12
2021-03-15


In [150]:
m = GEKKO()
X = []
X.append(m.Var(lb=1, ub=10))
X.append(m.Var(lb=1, ub=10))
X.append(m.Var(lb=1, ub=10))

m.Equation(X[2] >= X[1])

# X[0].value = 5
# X[1].value = 10
# X[2].value = 7


P = (W[0] + W[1]) / np.abs(1 + X[0] - X[1]) + (W[1] + W[2]) / np.abs(1 + X[1] - X[2]) + (W[0] + W[2]) / np.abs(1 + X[0] - X[2])
m.Obj(P)

m.solve()

apm 95.165.169.196_gk_model25 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            4
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              4
 Number of total equations: -            1
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              2
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program

In [103]:
W

[5, 8, 5]

In [155]:
X

[[825.78564173], [821.0], [830.0]]

In [69]:
W = [5,5,8]

In [72]:
W = [5,8,5]

In [147]:
X = [10, 1, 1]
m = 100
ind = [0,0,0]
eps = 1e-5
for i in range(1,11):
    for j in range(1,11):
        for k in range(1,11):
            if m > (W[0] + W[1]) / np.abs(eps + i - j) + (W[1] + W[2]) / np.abs(eps + j - k) + (W[0] + W[2]) / np.abs(eps + i - k):
                m = (W[0] + W[1]) / np.abs(eps + i - j) + (W[1] + W[2]) / np.abs(eps + j - k) + (W[0] + W[2]) / np.abs(eps + i - k)
                ind = [i,j,k]


In [ ]:
min(max)

In [128]:
from __future__ import print_function
import datetime
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']



"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    # if creds and creds.expired and creds.refresh_token:
    #     creds.refresh(Request())
    # else:
    flow = InstalledAppFlow.from_client_secrets_file(
        'credentials0.json', SCOPES)
    creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=986666376835-dqjk198rf1fhngi1f2hf8ios8mkipvej.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59560%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=0OtEDSlg4v0iiWeW5qcIpsLNQKNBJ3&access_type=offline
Getting the upcoming 10 events
No upcoming events found.


In [127]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [171]:
event = {
  'summary': 'Google I/O 2015',
  'description': 'A chance to hear more about Google\'s developer products.',
  'start': {
    'dateTime': '2015-05-28T00:00:00-07:00',
    'timeZone': 'Europe/Moscow',
  },
  'end': {
    'dateTime': '2015-05-28T12:00:00-07:00',
    'timeZone': 'Europe/Moscow',
  },
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

event = service.events().insert(calendarId='primary', body=event).execute()
print ('Event created: %s' % (event.get('htmlLink')))


Event created: https://www.google.com/calendar/event?eid=b2o4cDh1Z3Q5dTI5Y3IyYnMwcjE0ZjE4bTggZWdva290bzMxQG0


In [134]:
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

CREDENTIALS_FILE = 'credentials0.json'

def get_calendar_service():
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               CREDENTIALS_FILE, SCOPES)
           creds = flow.run_local_server(port=0)

       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('calendar', 'v3', credentials=creds)
   return service

In [11]:
calendar = {
    'summary': 'Календарь расписания дедлайнов',
    'timeZone': 'Europe/Moscow'
}

created_calendar = service.calendars().insert(body=calendar).execute()


calendars_result = service.calendarList().list().execute()

calendars = calendars_result.get('items', [])

calendar_id = calendars[-1]['id']

rule = {
    'scope': {
        'type': 'group',
        'value': 'dmitryokh0@gmail.com',
    },
    'role': 'writer'
}

created_rule = service.acl().insert(calendarId=calendar_id, body=rule).execute()


event = {
  'summary': 'Google I/O 2015',
  'description': 'A chance to hear more about Google\'s developer products.',
  'start': {
    'dateTime': '2015-05-28T00:00:00-07:00',
    'timeZone': 'Europe/Moscow',
  },
  'end': {
    'dateTime': '2015-05-28T12:00:00-07:00',
    'timeZone': 'Europe/Moscow',
  },
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

event = service.events().insert(calendarId='primary', body=event).execute()
print ('Event created: %s' % (event.get('htmlLink')))



Event created: https://www.google.com/calendar/event?eid=ampsNjI0dTZtamdwOG9iZGh1NGs1MTQwZDQgZWdva290bzMxQG0


In [32]:
calendars[-1]['id']

'lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com'

In [35]:
event.get('htmlLink')

'https://www.google.com/calendar/event?eid=ampsNjI0dTZtamdwOG9iZGh1NGs1MTQwZDQgZWdva290bzMxQG0'

In [34]:
service.events().insert(calendarId=calendars[-1]['id'], body=event).execute()

{'kind': 'calendar#event',
 'etag': '"3241389833915000"',
 'id': 'jjl624u6mjgp8obdhu4k5140d4',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=ampsNjI0dTZtamdwOG9iZGh1NGs1MTQwZDQgbHZmNDU4amVhcXFzZXZiZmg5bWU1bzFxdGtAZw',
 'created': '2021-05-11T01:01:56.000Z',
 'updated': '2021-05-11T01:01:57.001Z',
 'summary': 'Google I/O 2015',
 'description': "A chance to hear more about Google's developer products.",
 'creator': {'email': 'egokoto31@gmail.com'},
 'organizer': {'email': 'lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com',
  'displayName': 'Тестовый календарик',
  'self': True},
 'start': {'dateTime': '2015-05-28T07:00:00Z', 'timeZone': 'Europe/Moscow'},
 'end': {'dateTime': '2015-05-28T19:00:00Z', 'timeZone': 'Europe/Moscow'},
 'iCalUID': 'jjl624u6mjgp8obdhu4k5140d4@google.com',
 'sequence': 0,
 'reminders': {'useDefault': False,
  'overrides': [{'method': 'popup', 'minutes': 10},
   {'method': 'email', 'minutes': 1440}]},
 'eventType': 'default'}

In [137]:

service = get_calendar_service()
# Call the Calendar API
print('Getting list of calendars')
calendars_result = service.calendarList().list().execute()

calendars = calendars_result.get('items', [])

if not calendars:
   print('No calendars found.')
for calendar in calendars:
    summary = calendar['summary']
    id = calendar['id']
    primary = "Primary" if calendar.get('primary') else ""
    print("%s\t%s\t%s" % (summary, id, primary))

Getting list of calendars
Математический анализ 171 (ФКН ВШЭ) Высшая математика Высшая математика	classroom105581258328938939578@group.calendar.google.com	
Семья	family06039712825079928620@group.calendar.google.com	
ТВиМС 2018/2019, группа 171	classroom105762294756247308432@group.calendar.google.com	
Natural language processing	classroom117995594048403910684@group.calendar.google.com	
Праздники РФ	ru.russian#holiday@group.v.calendar.google.com	
Дни рождения	addressbook#contacts@group.v.calendar.google.com	
Производственный календарь	21u9bqthk7hed448vgu0l7ni6tr3t02f@import.calendar.google.com	
egokoto31@gmail.com	egokoto31@gmail.com	Primary
Тестовый календарик	tmb6sr07ube2hcoji6vh54mvik@group.calendar.google.com	
Тестовый календарик	lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com	
Тестовый календарик	27h91r2pisd5fr7o1a7thge86c@group.calendar.google.com	


In [45]:
service.calendarList().list().execute()

OSError: [Errno 49] Can't assign requested address

In [5]:
calendars_result = service.calendarList().list().execute()

calendars = calendars_result.get('items', [])

In [6]:
calendars[-1]

{'kind': 'calendar#calendarListEntry',
 'etag': '"1620695577405000"',
 'id': 'lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com',
 'summary': 'Тестовый календарик',
 'timeZone': 'UTC',
 'colorId': '3',
 'backgroundColor': '#f83a22',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [],
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [28]:
calendars_result.get('nextPageToken')

In [30]:
from icalendar import Calendar

ModuleNotFoundError: No module named 'icalendar'

In [29]:
Calendar(urlopen(url).read().decode('utf-8'))

NameError: name 'Calendar' is not defined

In [7]:
service.acl().get(calendarId='lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com').execute()

TypeError: Missing required parameter "ruleId"

In [ ]:
cal.to_ical()

In [8]:
service.acl().list(calendarId='lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com').execute()

{'kind': 'calendar#acl',
 'etag': '"p324etnk6mv0f00g"',
 'nextSyncToken': '00001620694917429000',
 'items': [{'kind': 'calendar#aclRule',
   'etag': '"00000000000000000000"',
   'id': 'user:lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com',
   'scope': {'type': 'user',
    'value': 'lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com'},
   'role': 'owner'},
  {'kind': 'calendar#aclRule',
   'etag': '"00001620680472974000"',
   'id': 'user:egokoto31@gmail.com',
   'scope': {'type': 'user', 'value': 'egokoto31@gmail.com'},
   'role': 'owner'}]}

In [143]:
calendar = {
    'summary': 'calendarSummary',
    'timeZone': 'America/Los_Angeles'
}

created_calendar = service.calendars().insert(body=calendar).execute()

print (created_calendar['id'])

p2ebn256dr7idnopc0gluuajus@group.calendar.google.com


In [146]:
created_calendar['id']

'p2ebn256dr7idnopc0gluuajus@group.calendar.google.com'

In [9]:
rule = {
    'scope': {
        'type': 'group',
        'value': 'egorkarus31@gmail.com',
    },
    'role': 'writer'
}

In [10]:
# rule = {
#     'scope': {
#         'type': 'scopeType',
#         'value': 'scopeEmail',
#     },
#     'role': 'writer'
# }

created_rule = service.acl().insert(calendarId='lvf458jeaqqsevbfh9me5o1qtk@group.calendar.google.com', body=rule).execute()

print (created_rule['id'])

user:egorkarus31@gmail.com


In [149]:
service.acl().inset

In [ ]:
https://calendar.google.com/calendar/ical/27h91r2pisd5fr7o1a7thge86c%40group.calendar.google.com/public/basic.ics
https://www.googleapis.com/calendar/v3/calendars/p2ebn256dr7idnopc0gluuajus/acl?alt=json

In [139]:
calendars[-1]

{'kind': 'calendar#calendarListEntry',
 'etag': '"1620680493659000"',
 'id': '27h91r2pisd5fr7o1a7thge86c@group.calendar.google.com',
 'summary': 'Тестовый календарик',
 'timeZone': 'Europe/Moscow',
 'colorId': '19',
 'backgroundColor': '#c2c2c2',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [],
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [142]:
service.calendars()

In [129]:
P2 = 0
X2 = [4,9,0]
for stud in range(len(student_deadlines)):
    lessons = []
    for i in range(len(student_deadlines[stud])):
        lessons = lessons + get_dead[student_deadlines[stud][i]]
#         print(get_dead[student_deadlines[stud][i]])
    print(lessons)
    for i in range(len(lessons)):
        for j in range(i+1, len(lessons)):
            P2 += (W[lessons[i]] + W[lessons[j]]) / np.abs(1e-2 + X2[lessons[i]] - X2[lessons[j]])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0, 1, 2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0, 1, 2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0, 1, 2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0, 1, 2]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [130]:
P2

104.66907670935962

In [100]:
tpr_date + datetime.timedelta(days=X[0][0])

datetime.date(2021, 4, 10)

In [101]:
tpr_date + datetime.timedelta(days=X[1][0])

datetime.date(2021, 4, 1)

In [102]:
tpr_date + datetime.timedelta(days=X[2][0])

datetime.date(2021, 4, 1)

In [323]:
P

0

In [298]:
tpr_date + 845

TypeError: unsupported operand type(s) for +: 'datetime.date' and 'int'

In [275]:
get_dead

{0: [0, 1, 2]}

In [255]:
subject

{0: [(10, '30.03.2021', '12.06.2021'),
  (6, '12.04.2021', '20.04.2021'),
  (9, '18.04.2021', '25.04.2021')]}

In [278]:
student_deadlines[12][0]

0

In [197]:
date_start

838

In [203]:
X

[0, 0, 0]

In [252]:
subject_id_to_name

{0: ('кр', 61045)}

In [282]:
P = 0
for stud in range(len(student_deadlines)):
    lessons = []
    for i in range(len(student_deadlines[stud])):
        lessons = lessons + get_dead[student_deadlines[stud][i]]
    
    for i in range(len(lessons)):
        for j in range(i+1, len(lessons)):
            P += (W[lessons[i]] + W[lessons[j]]) / np.abs(X[lessons[i]] - X[lessons[j]])

In [284]:
type(P)

gekko.gk_operators.GK_Operators

In [250]:
student_deadlines[12][0]

0

In [254]:
subject[0]

[(10, '30.03.2021', '12.06.2021'),
 (6, '12.04.2021', '20.04.2021'),
 (9, '18.04.2021', '25.04.2021')]

In [ ]:
for each student calc :
    sum_po_all_subject_he_in (вернуть все i которые в x[i] видимо):
        p+=(w[i] + w[j] / x[i] - x[j])

In [247]:
student_deadlines = []
for i in range(len(student_subjects)):
    tt = []
    for j in range(len(student_subjects[i])):
        try:
            tt = dedlines_by_lesson[student_subjects[i][j]]
        except:
            pass
    student_deadlines.append(tt)

In [246]:
dedlines_by_lesson

{61045: [0]}

In [248]:
student_deadlines

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0],
 [],
 [0],
 [],
 [],
 [0],
 [],
 [],
 [],
 [0],
 [],
 [0],
 [],
 [0],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [229]:
student_subjects[0]

[61043]

In [230]:
dedlines_by_lesson

{61045: [0]}

In [36]:
student_subjects = []
student_number = 0
for i in tqdm(range(len(fkn_ids))):
    url = 'https://ruz.hse.ru/api/schedule/student/' + str(fkn_ids[i]) + '?start=2021.01.01&finish=2021.06.15'
    ss = requests.get(url).json()
    time.sleep(1)
    sss = []
    for i in range(len(ss)):
        sss.append(ss[i]['disciplineOid'])
    sss = list(set(sss))
    student_subjects.append(sss)


NameError: name 'fkn_ids' is not defined

In [32]:

url = 'https://ruz.hse.ru/api/schedule/student/' + str(219088) + '?start=2021.01.01&finish=2021.06.15'
ss = requests.get(url).json()

In [33]:
ss

[]

In [285]:

m.Obj(P)

m.solve()

#Results
print('')
print('Results')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
# print('x4: ' + str(x4.value))

apm 95.165.169.196_gk_model13 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            5
   Intermediates:            0
   Connections  :            0
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:              5
 Number of total equations: -            2
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :              1
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program

In [159]:
student_subjects

{61043: 1,
 61061: 8,
 61062: 5,
 58665: 8,
 61046: 4,
 60317: 7,
 61064: 4,
 58642: 7,
 61045: 9,
 61528: 3,
 61050: 8,
 58653: 13,
 60133: 1,
 60134: 1,
 60855: 3,
 61047: 3,
 58666: 4,
 61042: 5,
 60311: 4,
 58279: 1,
 58287: 1,
 60850: 1,
 60853: 1,
 60710: 4,
 57830: 3,
 61069: 3,
 59414: 6,
 58649: 3,
 58152: 1,
 61086: 1,
 60315: 7,
 61051: 7,
 60897: 7,
 61026: 1,
 61065: 7,
 58379: 7,
 58577: 2,
 61367: 1,
 58376: 5,
 58921: 5,
 58571: 5,
 59765: 1,
 60886: 5,
 59419: 5,
 58589: 3,
 64206: 1,
 58685: 2,
 61038: 1,
 60320: 6,
 62625: 1,
 61032: 1,
 58623: 1,
 57858: 1,
 61027: 2,
 60842: 1,
 58587: 1,
 59529: 2,
 61388: 1,
 59537: 2,
 58016: 1,
 58600: 3,
 58605: 1,
 58606: 1,
 64302: 2,
 61070: 1,
 0: 5,
 60706: 3,
 58637: 3,
 64014: 5,
 58003: 3,
 58261: 3,
 61071: 5,
 63850: 1,
 61614: 2,
 58667: 5,
 61039: 5,
 61041: 5,
 63262: 2,
 61023: 2,
 60834: 1,
 59503: 1,
 61434: 2,
 61063: 1,
 61068: 2,
 58679: 2,
 58680: 2,
 58604: 1,
 61368: 1,
 59584: 2,
 58611: 1}

In [ ]:
for in range(len)

In [64]:
subject[0][2][2].split('.')[0]

'32.23.32.'

In [160]:
subject

{0: [(7, 'ф', 'вфа'), (7, 'уа', 'уа'), (4, '12.12.12', '32.23.32.')],
 1: [(4, 'ц', 'ц')],
 2: [(7, 'аыв', 'авы')],
 3: []}

In [53]:

>>> a = date(2011,11,24)
>>> b = date(2011,11,17)
>>> a-b
datetime.timedelta(7)
>>> (a-b).days

NameError: name 'datetime' is not defined

In [38]:
import numpy as np
from gekko import GEKKO
m = GEKKO()
ni = 8
nj = 3
x = [[m.Var(lb=0,integer=True) for j in range(nj)] for i in range(ni)]
s = 0
for i in range(ni):
    for j in range(nj):
        s += x[i][j]
m.Equation(s==10)
m.Equations([x[2][j]+x[4][j]>=x[0][j] for j in range(nj)])
m.Equations([x[3][j]+x[5][j]>=x[1][j] for j in range(nj)])
for j in range(nj):
    x[6][j].upper=15
    x[7][j].upper=15
m.Equations([(m.sign3(x[6][j])==m.sign3(x[2][j])) for j in range(nj)])
m.Equations([(m.sign3(x[7][j])==m.sign3(x[3][j])) for j in range(nj)])
m.Obj(sum([m.tanh(x[i][0]) for i in range(ni)]))
m.options.SOLVER=1
m.solve()
print(x)

apm 95.165.169.196_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           78
   Intermediates:            0
   Connections  :            0
   Equations    :           50
   Residuals    :           50
 
 Number of state variables:             78
 Number of total equations: -           49
 Number of slack variables: -           30
 ---------------------------------------
 Degrees of freedom       :             -1
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj:  0.00E+00 Gap:       NaN
Iter:     2 I: -

In [52]:
m = GEKKO()
x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]


m.Equation(x1+x2+x3+x4<=25)

m.Obj(-1*(x1**2 + x2**2 + x3**2 + x4**2))

m.solve()

#Results
print('')
print('Results')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
print('x4: ' + str(x4.value))

apm 95.165.169.196_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            5
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              5
 Number of total equations: -            1
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              3
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program 

In [48]:
m = GEKKO()

#define parameter
eq = m.Param(value=40)

#initialize variables
x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]

#initial values
x1.value = 1
x2.value = 5
x3.value = 5
x4.value = 1

#Equations
m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==eq)

#Objective
m.Obj(x1*x4*(x1+x2+x3)+x3)

#Set global options
m.options.IMODE = 3 #steady state optimization

#Solve simulation
m.solve()

#Results
print('')
print('Results')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
print('x4: ' + str(x4.value))

apm 95.165.169.196_gk_model1 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            6
   Intermediates:            0
   Connections  :            0
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:              5
 Number of total equations: -            2
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              2
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [42]:
ni = 8
nj = 3
x = [[m.Var(lb=0,integer=True) for j in range(nj)] for i in range(ni)]
s = 0

In [43]:
for i in range(ni):
    for j in range(nj):
        s += x[i][j]

In [45]:
j

2

In [46]:
m.Equation(s==10)
m.Equations([x[2][j]+x[4][j]>=x[0][j] for j in range(nj)])
m.Equations([x[3][j]+x[5][j]>=x[1][j] for j in range(nj)])

In [47]:
m

In [40]:
[[m.Var(lb=0,integer=True) for j in range(nj)] for i in range(ni)]

[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [41]:
x

[[[0.0], [2.0], [1.0]],
 [[0.0], [1.0], [1.0]],
 [[0.0], [0.0], [0.0]],
 [[0.0], [0.0], [0.0]],
 [[0.0], [2.0], [1.0]],
 [[0.0], [1.0], [1.0]],
 [[0.0], [0.0], [0.0]],
 [[0.0], [0.0], [0.0]]]

In [24]:
dedline

{(61045, 0): (10, 'ацувс', 'цавыс'),
 (63510, 0): (8, '12.12.2021', '20.12.2021')}

In [18]:
cur_ded

{341821214: (61045, 1)}

In [19]:
cur_discipline

{341821214: 61045}

In [91]:
dedline

{(61045, 0): (0, 0, 0)}

In [7]:
state

{341821214: 15}

In [100]:
dedline[cur_ded[341821214]] = (5,5,dedline[cur_ded[341821214]][0])

In [101]:
dedline[cur_ded[341821214]]

(5, 5, 0)

d
пми
a5965e4d89-2d4c0f4ba-6580f3b-3ddfd0-3c6
Назад
a5965e4d89-2d4c0f4ba-6580f3b-3ddfd0-3c6
Евгений
Соколов
Андреевич


Unhandled exception on callback
Traceback (most recent call last):
  File "/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/telethon/client/updates.py", line 454, in _dispatch_update
    await callback(event)
  File "<ipython-input-36-53c9dde30abf>", line 256, in callback
    cur_op_in_teacher[event.chat.id] = course[event.message.message]
AttributeError: 'Event' object has no attribute 'message'


CancelledError: 

In [16]:
dedline

{(61045, 0): (4, 'ц', 'у'), (61045, 1): (0, 'аыц', 'ывс')}

In [54]:
str(len(subs))

NameError: name 'subs' is not defined

In [16]:
int(str(t).split("'")[1])

63548

In [261]:
name = 'Колодеева Ольга'

In [263]:
name.split(' ')

['Колодеева', 'Ольга']

In [337]:
surname = 'Колодеева'
name = 'Ольга'
last_name = ''

In [69]:
surname = 'Сибирякова'
name = 'Ольга'
last_name = 'Викторовна'

In [362]:
surname = 'Соколов'
name = 'Евгений'
last_name = 'Андреевич'

In [431]:
url_id = 'https://ruz.hse.ru/api/search?term=' + 'Котов' + '%' + 'Егор' + '%' + 'Александрович'
soup_id = requests.get(url_id).json()
ids_me = []
for i in range(len(soup_id)):
    if soup_id[i]['type'] == 'student':
        ids_me.append(soup_id[i]['id'])
url = 'https://ruz.hse.ru/api/schedule/student/' + ids_me[0] + '?start=2021.01.01&finish=2021.06.11'
data = requests.get(url).json()
my_disciplines = {}
for mini_data in data:
    if mini_data['discipline'] not in list(disciplines.values()):
        my_disciplines[mini_data['disciplineOid']] = mini_data['discipline']

In [432]:
my_disciplines

{61061: 'Машинное обучение для больших данных (рус)',
 58665: 'Научно-исследовательский семинар "Машинное обучение и приложения 2" (рус)',
 61051: 'Обучение с подкреплением (рус)',
 61046: 'Компьютерное зрение (рус)',
 60317: 'Соревновательный анализ данных (анг)'}

In [416]:
soup_id

[{'id': '216597',
  'label': 'Котов Егор Александрович',
  'description': 'Бакалавриат группа БПМИ172-2017',
  'type': 'student'}]

In [ ]:
discipline
disciplineOid

In [431]:
requests.get(url).json()[0]

{'auditorium': 'Онлайн',
 'auditoriumAmount': 100,
 'auditoriumOid': 4803,
 'author': 'STAFF\\mgolovina',
 'beginLesson': '09:30',
 'building': 'Вавилова ул., д. 7',
 'buildingGid': 55,
 'buildingOid': 2174,
 'contentOfLoadOid': 946912,
 'contentOfLoadUID': '4288795551',
 'contentTableOfLessonsName': '1',
 'contentTableOfLessonsOid': 2334,
 'createddate': '2020-08-22T15:38:37Z00:00',
 'date': '2021.04.05',
 'dateOfNest': '/Date(1617570000000+0300)/',
 'dayOfWeek': 1,
 'dayOfWeekString': 'Пн',
 'detailInfo': '',
 'discipline': 'Зоология позвоночных (рус)',
 'disciplineOid': 63548,
 'disciplineinplan': '4084126628',
 'disciplinetypeload': 7,
 'duration': 2,
 'endLesson': '10:50',
 'group': 'ЗоолПозв_Б2020_кимб_2_1#С#Зоология позвоночных',
 'groupOid': 41894,
 'groupUID': '4125201762',
 'group_facultyoid': 8248,
 'hideincapacity': 0,
 'isBan': False,
 'kindOfWork': 'Практическое занятие',
 'kindOfWorkOid': 964,
 'kindOfWorkUid': '1861272403',
 'lecturer': 'доц. Сибирякова Ольга Викторовна

In [482]:
id = '48639'
url = 'https://ruz.hse.ru/api/schedule/lecturer/' + id + '?start=2021.04.05&finish=2021.04.11'
requests.get(url).json()

[{'auditorium': 'Онлайн',
  'auditoriumAmount': 100,
  'auditoriumOid': 4803,
  'author': 'STAFF\\mgolovina',
  'beginLesson': '09:30',
  'building': 'Вавилова ул., д. 7',
  'buildingGid': 55,
  'buildingOid': 2174,
  'contentOfLoadOid': 946912,
  'contentOfLoadUID': '4288795551',
  'contentTableOfLessonsName': '1',
  'contentTableOfLessonsOid': 2334,
  'createddate': '2020-08-22T15:38:37Z00:00',
  'date': '2021.04.05',
  'dateOfNest': '/Date(1617570000000+0300)/',
  'dayOfWeek': 1,
  'dayOfWeekString': 'Пн',
  'detailInfo': '',
  'discipline': 'Зоология позвоночных (рус)',
  'disciplineOid': 63548,
  'disciplineinplan': '4084126628',
  'disciplinetypeload': 7,
  'duration': 2,
  'endLesson': '10:50',
  'group': 'ЗоолПозв_Б2020_кимб_2_1#С#Зоология позвоночных',
  'groupOid': 41894,
  'groupUID': '4125201762',
  'group_facultyoid': 8248,
  'hideincapacity': 0,
  'isBan': False,
  'kindOfWork': 'Практическое занятие',
  'kindOfWorkOid': 964,
  'kindOfWorkUid': '1861272403',
  'lecturer':

In [73]:
requests.get(url_id).json()[0]['id']

{'id': '243826',
 'label': 'Колодеева Ольга -',
 'description': 'Бакалавриат группа МКМБ202-2020',
 'type': 'student'}

In [393]:
soup_id

[{"id":"48639", "label":"Сибирякова Ольга Викторовна", "description":"базовая кафедра Института биоорганической химии им. академиков М.М. Шемякина и Ю.А. Овчинникова РАН", "type":"lecturer"},{"id":"3857959631", "label":"Сибирякова Ольга Викторовна", "description":"базовая кафедра Института биоорганической химии им. академиков М.М. Шемякина и Ю.А. Овчинникова РАН", "type":"person"}]

In [78]:
requests.get(url).json()[0]['disciplineOid']

63548

In [160]:

url_id = 'https://ruz.hse.ru/api/search?term=Котов%Егор%Александрович&type=student'
id = requests.get(url_id).json()[0]['id']
url = 'https://ruz.hse.ru/api/schedule/student/' + id + '?start=2021.01.01&finish=2021.06.10'

ss = requests.get(url).json()


In [163]:
url = 'https://ruz.hse.ru/api/schedule/student/216597?start=2021.01.01&finish=2021.05.11'

In [164]:
requests.get(url).json()

[{'auditorium': 'Online (ауд бронь 52)',
  'auditoriumAmount': 100,
  'auditoriumOid': 5723,
  'author': 'STAFF\\makrylova',
  'beginLesson': '11:10',
  'building': 'Покровский б-р, д.11',
  'buildingGid': 0,
  'buildingOid': 2311,
  'contentOfLoadOid': 988977,
  'contentOfLoadUID': '4420547737',
  'contentTableOfLessonsName': '2',
  'contentTableOfLessonsOid': 2335,
  'createddate': '2020-12-16T21:39:33Z00:00',
  'date': '2021.01.12',
  'dateOfNest': '/Date(1610398800000+0300)/',
  'dayOfWeek': 2,
  'dayOfWeekString': 'Вт',
  'detailInfo': '',
  'discipline': 'Машинное обучение для больших данных (рус)',
  'disciplineOid': 61061,
  'disciplineinplan': '3855325353',
  'disciplinetypeload': 7,
  'duration': 2,
  'endLesson': '12:30',
  'group': 'МОдБД_Б2017_ПМИИ_2#Г#Машинное обучение для больших данных',
  'groupOid': 23513,
  'groupUID': '3860048663',
  'group_facultyoid': 5588,
  'hideincapacity': 0,
  'isBan': False,
  'kindOfWork': 'Семинар',
  'kindOfWorkOid': 960,
  'kindOfWorkUid

[]

In [158]:
student_subjects

{61043: 1,
 61061: 8,
 61062: 5,
 58665: 8,
 61046: 4,
 60317: 7,
 61064: 4,
 58642: 7,
 61045: 9,
 61528: 3,
 61050: 8,
 58653: 13,
 60133: 1,
 60134: 1,
 60855: 3,
 61047: 3,
 58666: 4,
 61042: 5,
 60311: 4,
 58279: 1,
 58287: 1,
 60850: 1,
 60853: 1,
 60710: 4,
 57830: 3,
 61069: 3,
 59414: 6,
 58649: 3,
 58152: 1,
 61086: 1,
 60315: 7,
 61051: 7,
 60897: 7,
 61026: 1,
 61065: 7,
 58379: 7,
 58577: 2,
 61367: 1,
 58376: 5,
 58921: 5,
 58571: 5,
 59765: 1,
 60886: 5,
 59419: 5,
 58589: 3,
 64206: 1,
 58685: 2,
 61038: 1,
 60320: 6,
 62625: 1,
 61032: 1,
 58623: 1,
 57858: 1,
 61027: 2,
 60842: 1,
 58587: 1,
 59529: 2,
 61388: 1,
 59537: 2,
 58016: 1,
 58600: 3,
 58605: 1,
 58606: 1,
 64302: 2,
 61070: 1,
 0: 5,
 60706: 3,
 58637: 3,
 64014: 5,
 58003: 3,
 58261: 3,
 61071: 5,
 63850: 1,
 61614: 2,
 58667: 5,
 61039: 5,
 61041: 5,
 63262: 2,
 61023: 2,
 60834: 1,
 59503: 1,
 61434: 2,
 61063: 1,
 61068: 2,
 58679: 2,
 58680: 2,
 58604: 1,
 61368: 1,
 59584: 2,
 58611: 1}

In [151]:
import time
time.sleep(5)

In [156]:
student_subjects = {}
for i in tqdm(range(len(fkn_ids))):
    url = 'https://ruz.hse.ru/api/schedule/student/' + str(fkn_ids[i]) + '?start=2021.01.01&finish=2021.06.15'
    ss = requests.get(url).json()
    time.sleep(1)
    sss = []
    for i in range(len(ss)):
        sss.append(ss[i]['disciplineOid'])
    sss = list(set(sss))
    for i in range(len(sss)):
        if sss[i] in student_subjects:
            student_subjects[sss[i]] += 1
        else:
            student_subjects[sss[i]] = 1


  4%|▍         | 96/2457 [01:52<46:03,  1.17s/it]


KeyboardInterrupt: 

In [155]:
ss

[]

In [157]:
student_subjects

{61043: 1,
 61061: 8,
 61062: 5,
 58665: 8,
 61046: 4,
 60317: 7,
 61064: 4,
 58642: 7,
 61045: 9,
 61528: 3,
 61050: 8,
 58653: 13,
 60133: 1,
 60134: 1,
 60855: 3,
 61047: 3,
 58666: 4,
 61042: 5,
 60311: 4,
 58279: 1,
 58287: 1,
 60850: 1,
 60853: 1,
 60710: 4,
 57830: 3,
 61069: 3,
 59414: 6,
 58649: 3,
 58152: 1,
 61086: 1,
 60315: 7,
 61051: 7,
 60897: 7,
 61026: 1,
 61065: 7,
 58379: 7,
 58577: 2,
 61367: 1,
 58376: 5,
 58921: 5,
 58571: 5,
 59765: 1,
 60886: 5,
 59419: 5,
 58589: 3,
 64206: 1,
 58685: 2,
 61038: 1,
 60320: 6,
 62625: 1,
 61032: 1,
 58623: 1,
 57858: 1,
 61027: 2,
 60842: 1,
 58587: 1,
 59529: 2,
 61388: 1,
 59537: 2,
 58016: 1,
 58600: 3,
 58605: 1,
 58606: 1,
 64302: 2,
 61070: 1,
 0: 5,
 60706: 3,
 58637: 3,
 64014: 5,
 58003: 3,
 58261: 3,
 61071: 5,
 63850: 1,
 61614: 2,
 58667: 5,
 61039: 5,
 61041: 5,
 63262: 2,
 61023: 2,
 60834: 1,
 59503: 1,
 61434: 2,
 61063: 1,
 61068: 2,
 58679: 2,
 58680: 2,
 58604: 1,
 61368: 1,
 59584: 2,
 58611: 1}

In [149]:

url = 'https://ruz.hse.ru/api/schedule/student/' + str(221105) + '?start=2021.01.01&finish=2021.06.15'
student_subjects = {}
ss = requests.get(url).json()
for i in range(len(ss)):
    if ss[i]['disciplineOid'] in student_subjects:
        student_subjects[ss[i]['disciplineOid']] += 1
    else:
        student_subjects[ss[i]['disciplineOid']] = 1
student_subjects

{60320: 60, 58637: 36, 64014: 7, 61068: 39, 59384: 32, 58261: 35, 0: 7}

In [148]:
student_subjects

{}

In [142]:
len(fkn_ids)

2457

In [94]:
fkn_ids

[212192,
 212204,
 212216,
 212229,
 212230,
 212249,
 212252,
 212272,
 212282,
 212286,
 212293,
 212297,
 212299,
 212305,
 212316,
 212319,
 212324,
 212326,
 212328,
 212335,
 212340,
 212342,
 212343,
 212352,
 212354,
 212360,
 212365,
 212366,
 212379,
 212395,
 212396,
 212398,
 212404,
 212405,
 212425,
 212429,
 212430,
 212431,
 212445,
 212447,
 212449,
 212451,
 212454,
 212458,
 212469,
 212482,
 212490,
 212496,
 212500,
 212510,
 212523,
 212526,
 212531,
 212536,
 212551,
 212562,
 212565,
 212568,
 212570,
 212578,
 212592,
 212595,
 212599,
 212600,
 212617,
 212620,
 212648,
 212649,
 212655,
 212656,
 212659,
 212661,
 212667,
 212703,
 212710,
 212716,
 212720,
 212722,
 212733,
 212734,
 212755,
 212756,
 212766,
 212772,
 212783,
 212786,
 212801,
 212805,
 212806,
 212809,
 212813,
 212820,
 212821,
 212826,
 212827,
 212839,
 212842,
 212843,
 212851,
 212854,
 212868,
 212871,
 212872,
 212885,
 212886,
 212893,
 212899,
 212905,
 212911,
 212925,
 212943,
 

[]

In [133]:
import sys
!{sys.executable} -m pip install cal_setup

ERROR: Could not find a version that satisfies the requirement cal_setup
ERROR: No matching distribution found for cal_setup
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.12/libexec/bin/python3.9 -m pip install --upgrade pip' command.


In [278]:
soup = BeautifulSoup(requests.get(url).content, "html.parser")

In [279]:
soup

[{"auditorium":"D508","auditoriumAmount":56,"auditoriumOid":4580,"author":"STAFF\\makrylova","beginLesson":"13:00","building":"Покровский б-р, д.11","buildingGid":0,"buildingOid":2311,"contentOfLoadOid":893516,"contentOfLoadUID":"3871399209","contentTableOfLessonsName":"3","contentTableOfLessonsOid":2336,"createddate":"2020-08-23T22:48:22Z00:00","date":"2021.04.06","dateOfNest":"\/Date(1617656400000+0300)\/","dayOfWeek":2,"dayOfWeekString":"Вт","detailInfo":"","discipline":"Теория и практика многопоточной синхронизации (рус)","disciplineOid":61069,"disciplineinplan":"3855319144","disciplinetypeload":7,"duration":2,"endLesson":"14:20","group":"ТиПМС_Б2018_ПМИИ#П#Теория и практика многопоточной синхронизации","groupOid":23537,"groupUID":"3860141595","group_facultyoid":5588,"hideincapacity":0,"isBan":false,"kindOfWork":"Лекция","kindOfWorkOid":914,"kindOfWorkUid":"401830420","lecturer":"ст.преп. Липовский Роман Германович","lecturerCustomUID":null,"lecturerEmail":"rlipovskiy@hse.ru","lect

In [277]:
url = 'https://ruz.hse.ru/api/schedule/student/212565?start=2021.04.05&finish=2021.04.11'

In [66]:
import pandas as pd
import html5lib
import BeautifulSoup4
# f_states=   pd.read_html('https://ruz.hse.ru/api/schedule/student/212565?start=2021.04.05&finish=2021.04.11')

ModuleNotFoundError: No module named 'BeautifulSoup4'

In [67]:
import json

In [247]:
a = json.loads(a)

In [257]:
url_id = 'https://ruz.hse.ru/api/search?term=Колодеева%Ольга%&type=student'

In [ ]:
1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU
1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms

In [297]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU'
SAMPLE_RANGE_NAME = 'Class Data!A2:E'


"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

if not values:
    print('No data found.')
else:
    print('Name, Major:')
    for row in values:
        # Print columns A and E, which correspond to indices 0 and 4.
        print('%s, %s' % (row[0], row[4]))


FileNotFoundError: [Errno 2] No such file or directory

In [318]:
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'yego-815@my-project-dimpoma.iam.gserviceaccount.com'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

NameError: name 'spreadsheetId' is not defined

In [ ]:
service.spreadsheets execute No such file or directory

In [324]:
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

CREDENTIALS_FILE = 'credentials2.json'  # Имя файла с закрытым ключом, вы должны подставить свое

# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Первый тестовый документ', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # сохраняем идентификатор файла
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

FileNotFoundError: [Errno 2] No such file or directory

In [335]:
from googleapiclient.discovery import build  # Added
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/drive']  # Modified
credentials = service_account.Credentials.from_service_account_file('credentials4.json', scopes=SCOPES)

drive = build('drive', 'v3', credentials=credentials)
file_metadata = {
    'name': 'sampleName',
    'parents': ['#### folderId ###'],
    'mimeType': 'application/vnd.google-apps.spreadsheet',
}
res = drive.files().create(body=file_metadata).execute()
print(res)

FileNotFoundError: [Errno 2] No such file or directory

In [339]:

import gspread

In [362]:
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Первый тестовый документ', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # сохраняем идентификатор файла
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

FileNotFoundError: [Errno 2] No such file or directory

In [357]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials2.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

In [358]:
sheet = client.open('Таблица')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

In [359]:
sheet_instance

<Worksheet 'Лист1' id:0>

In [361]:
5

5

In [360]:
import gspread
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials2.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
#Now will can access our google sheets we call client.open on StartupName
sheet = client.open('Таблица').sheet1
pp = pprint.PrettyPrinter()
results = sheet.get_all_records()
results

[{'sf': 'dfg'}, {'sf': 'dfg'}]

In [49]:
ans[0]

('кр-1',
 datetime.date(2021, 4, 1),
 'Машинное обучение для больших данных (рус)')

In [57]:
ans[0] = (ans[0][0], str(ans[0][1]), ans[0][2])

In [53]:
list(ans[0])

['кр-1',
 datetime.date(2021, 4, 1),
 'Машинное обучение для больших данных (рус)']

In [59]:
import gspread
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials2.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
#Now will can access our google sheets we call client.open on StartupName
sheet = client.open('Таблица').sheet1
pp = pprint.PrettyPrinter()
results = sheet.get_all_records()
results

sheet.append_row(list(ans[0]))

{'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
 'tableRange': "'Лист1'!A1:C5",
 'updates': {'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
  'updatedRange': "'Лист1'!A6:C6",
  'updatedRows': 1,
  'updatedColumns': 3,
  'updatedCells': 3}}

In [378]:
X

[[750.00000123],
 [758.00008078],
 [766.00000644],
 [780.60274715],
 [788.08751061],
 [796.16779887],
 [812.06469371],
 [819.0],
 [773.06358349],
 [804.45811585]]

In [379]:
for i in range(len(X)):
    print(tpr_date + datetime.timedelta(days=X[i][0]))

2021-01-20
2021-01-28
2021-02-05
2021-02-19
2021-02-27
2021-03-07
2021-03-23
2021-03-30
2021-02-12
2021-03-15


In [377]:
sheet.append_row([1,2])

{'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
 'tableRange': "'Лист1'!A1:A3",
 'updates': {'spreadsheetId': '1VncWFoiz9xci7KiRP2ZNnM74D9fxTc-0ZElbDbRm0VU',
  'updatedRange': "'Лист1'!A4:B4",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [375]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START sheets_quickstart]
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
SAMPLE_RANGE_NAME = 'Class Data!A2:E'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials4.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        print('Name, Major:')
        for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
            print('%s, %s' % (row[0], row[4]))

if __name__ == '__main__':
    main()
# [END sheets_quickstart]


RefreshError: ('deleted_client: The OAuth client was deleted.', {'error': 'deleted_client', 'error_description': 'The OAuth client was deleted.'})

In [371]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = '1cvZswLiDo3LfhnA7RcS8vFqacx73RGor-OZ_FtvyLE8'
SAMPLE_RANGE_NAME = 'A1:AA1000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials2.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

df=pd.DataFrame(values_input[1:], columns=values_input[0])

ValueError: Client secrets must be for a web or installed app.

In [366]:
rows = SHEETS.spreadsheets().values().get(spreadsheetId=SHEET_ID,
    range='Sheet1').execute().get('values', [])

NameError: name 'SHEETS' is not defined

In [347]:
client.open('https://docs.google.com/spreadsheets/d/1ww2UcaIz19H_1MRJLmCLp6UGQMvg1_xg65yrn0uZ0rQ')


SpreadsheetNotFound: 

In [331]:
file_metadata = {
'name' : 'Invoices',
'mimeType' : 'application/vnd.google-apps.folder'
}
file = drive_service.files().create(body=file_metadata,
fields='id').execute()

NameError: name 'drive_service' is not defined

In [364]:
spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Таблица', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()

FileNotFoundError: [Errno 2] No such file or directory

In [314]:
spreadsheet.execute()

FileNotFoundError: [Errno 2] No such file or directory

In [322]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START sheets_quickstart]
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
SAMPLE_RANGE_NAME = 'Class Data!A2:E'

"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

if not values:
    print('No data found.')
else:
    print('Name, Major:')
    for row in values:
        # Print columns A and E, which correspond to indices 0 and 4.
        print('%s, %s' % (row[0], row[4]))


# [END sheets_quickstart]

RefreshError: ('invalid_scope: Bad Request', {'error': 'invalid_scope', 'error_description': 'Bad Request'})

In [317]:
creds

In [289]:
!cd /usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/

In [292]:

import certifi
certifi.where()

'/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/certifi/cacert.pem'

In [288]:
/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/requests/cacert.pem
/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/certifi/cacert.pem

SyntaxError: invalid syntax (<ipython-input-288-83ac1666e8ec>, line 1)

In [286]:
import sys, os


def override_where():
    """ overrides certifi.core.where to return actual location of cacert.pem"""
    # change this to match the location of cacert.pem
    return os.path.abspath("cacert.pem")


# is the program compiled?
if hasattr(sys, "frozen"):
    import certifi.core

    os.environ["REQUESTS_CA_BUNDLE"] = override_where()
    certifi.core.where = override_where

    # delay importing until after where() has been replaced
    import requests.utils
    import requests.adapters
    # replace these variables in case these modules were
    # imported before we replaced certifi.core.where
    requests.utils.DEFAULT_CA_BUNDLE_PATH = override_where()
    requests.adapters.DEFAULT_CA_BUNDLE_PATH = override_where()

In [282]:
InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)